# 永磁同步电机-场路耦合-电机性能计算

本notebook说明并验证了永磁同步电机(PMSM)的性能计算方法，包括：
- 电压/电流限幅计算
- 电磁转矩计算
- 电角速度计算
- MTPA（最大转矩/电流比）控制轨迹计算
- MTPV（最大转矩/功率比）控制轨迹计算
- 等转矩线计算
- T-n（转矩-转速）曲线计算
···

## 0 常量说明

| 常量名 | 类型 | 说明 |
| ------ | ---- | ---- |
| id0Points | int | id=0控制策略iq扫描点数 |
| mtpaPoints | int | MTPA曲线扫描点数 |
| idPointsAtICircle | int | 定子电流圆id扫描点数 |
| deMagidPoints | int | 去磁电流迭代初始等分数 |
| deMagidTol | float | d轴电流迭代允许误差 |
| deMagidMaxIter | int | d轴电流最大迭代次数 |
| equUPoints | int | 等电压椭圆iq扫描点数 |
| mtpvWeScanRatio | int | MTPV转速扫描倍数(相对基速，电角速度) |
| mtpvWePoints | int | MTPV转速扫描范围等分点数 |
| fw1iqPoints | int | 弱磁I区iq扫描点数 |
| equTemIsPoints | int | 指定转矩，计算id、id的初始等分数 |
| equTemPoints | int | 等转矩线扫描点数 |
| equTemTracksNum | int | 等转矩线数量 |
| constTemPointsAtNonFW | int | 非弱磁区的恒转矩点数 |
| constWePointsAtNonFW | int | 非弱磁区的恒电角速度点数 |
| constTemPointsAtFW | int | 弱磁区的恒转矩点数 |
| constWePointsAtFW | int | 弱磁区的恒电角速度点数 |

In [ ]:
# 常量
id0Points      = 200             # id=0控制策略iq扫描点数
mtpaPoints     = 200             # MTPA曲线扫描点数
idPointsAtICircle = 1000          # 定子电流圆id扫描点数
deMagidPoints  = 10               # 去磁电流迭代初始等分数
deMagidTol     = 1e-5             # d轴电流迭代允许误差
deMagidMaxIter = 10              # d轴电流最大迭代次数
equUPoints       = 2000             # 等电压椭圆iq扫描点数
mtpvWeScanRatio = 20             # MTPV转速扫描倍数(相对基速，电角速度)
mtpvWePoints   = 200              # MTPV转速扫描点数
fw1iqPoints    = 200              # 弱磁I区iq扫描点数
equTemIsPoints  = 300              # 指定转矩，计算id、iq的初始等分数
equTemPoints    = 100              # 等转矩线扫描点数
equTemTracksNum = 30               # 等转矩线数量
constTemPointsAtNonFW = 100        # 非弱磁区的恒转矩点数
constWePointsAtNonFW = 100            # 非弱磁区的恒电角速度点数
constTemPointsAtFW = 200            # 弱磁区的恒转矩点数
constWePointsAtFW = 200            # 弱磁区的恒电角速度点数
IsSmoothInterp = True            # 是否使用三次插值

## 1 输入参数

| 参数名 | 类型 | 说明 |
| ------ | ---- | ---- |
| PsiR_vs_id | np.ndarray (N×2) | 磁链-d轴电流曲线，第1列为id，第2列为PsiR |
| Lad_vs_id | np.ndarray (N×2) | d轴激磁电感-d轴电流曲线，第1列为id，第2列为Ld |
| Laq_vs_iq | np.ndarray (N×2) | q轴激磁电感-q轴电流曲线，第1列为iq，第2列为Lq |
| Udc | float | 变频器母线电压 |
| Imax | float | 变频器限幅电流(线电流幅值) |
| connectType | str | 绕组连接方式，取值: 'Y' 或 'D' |
| modulationType | str | 调制方式，取值: 'SVPWM' 或 'SPWM' |
| polePairs | int | 极对数 |
| Lsigma | float | 定子漏抗 |
| Rs | float | 定子相电阻 |
| We_base | float | 基度 [rad/s] |

In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号

print("库导入成功！")

In [ ]:
# 设置测试用 PMSM 电机参数

# 利用二次函数，生成更充足的点集

# 设置更细致的取值范围
id_range = np.linspace(-300, 100, 61)  # 61点, -200到100A
iq_range = np.linspace(-300, 300, 71)  # 71点, -300到300A

# 假设磁链-id特性/电感用二次曲线近似（实际可参数拟合更精细）
# 磁链 Ψr 随 d轴电流 id 的二次下降
PsiR_vs_id = np.column_stack([
	id_range,
	1.0 - 2.0e-4 * id_range - 2.5e-7 * id_range**2
])  # 1.0 - k1*id - k2*id^2

# d轴电感 Ld 随 id 减小而略升高，用以顶点表达的二次曲线模拟
# 顶点位置 id0 = -133.33A，Ld0 = 0.648888mH
# Ld = Ld0 + a*(id - id0)^2，a = 1.5e-8
id0 = -133.33  # 顶点位置
Ld0 = 0.006  # 顶点时Ld的取值（H）
a = -1.5e-8  # 二次项系数
Lad_vs_id = np.column_stack([
	id_range,
	Ld0 + a * (id_range - id0) ** 2
])

# q轴电感 Lq 对iq=0“钟形”二次分布
Laq_vs_iq = np.column_stack([
	iq_range,
	9e-3 - 8.0e-9 * iq_range**2  # “钟形”二次分布
])

# 额定参数
Udc            = 540.0           # 变频器母线电压 [V]
Imax           = 300.0            # 变频器限幅电流(线电流幅值) [A]
connectType    = 'Y'             # 绕组连接方式：'Y'型
modulationType = 'SVPWM'         # 调制方式
polePairs      = 4               # 极对数
Lsigma         = 0.015e-3         # 谐波漏抗 [H]
Rs             = 0.02            # 相电阻 [Ω]
We_base        = 100.0      # 基准电角速度 [rad/s]
print("测试参数初始化完成！")

In [ ]:
# 修改标志位，判断是否使用常数磁链和电感测试数据
use_const_L_param = False  # True: 使用常数磁链和电感；False: 用默认曲线

if use_const_L_param:
	# 额外测试：生成磁链、电感常数序列（无随电流变化）
	# 取值范围
	id_range = np.linspace(-300, 100, 100)  # 100点, -200到100A
	iq_range = np.linspace(-300, 300, 100)  # 100点, -300到300A

	# 常数磁链和电感（测试用例）
	PsiR_const = 1.0   # Wb，常数磁链
	Ld_const = 5.7e-3   # H，d轴电感常数
	Lq_const = 9e-3     # H，q轴电感常数

	# 构造常数磁链曲线
	PsiR_vs_id = np.column_stack([
		id_range,
		np.full_like(id_range, PsiR_const)
	])

	# 构造d轴电感常数曲线
	Lad_vs_id = np.column_stack([
		id_range,
		np.full_like(id_range, Ld_const)
	])

	# 构造q轴电感常数曲线
	Laq_vs_iq = np.column_stack([
		iq_range,
		np.full_like(iq_range, Lq_const)
	])
	print("固定常数磁链、电感数据序列生成完成！")

In [ ]:
# 绘制磁链-d轴电流曲线
plt.figure(figsize=(14, 10))

plt.subplot(3, 1, 1)
plt.plot(PsiR_vs_id[:, 0], PsiR_vs_id[:, 1], marker='o', label='Ψr-id')
plt.xlabel("id (A)")
plt.ylabel("Ψr (Wb)")
plt.title("磁链-id 曲线")
plt.grid(True)
plt.legend()

# 绘制d轴电感-d轴电流曲线
plt.subplot(3, 1, 2)
plt.plot(Lad_vs_id[:, 0], Lad_vs_id[:, 1] * 1e3, marker='s', color='orange', label='Ld-id')
plt.xlabel("id (A)")
plt.ylabel("Ld (mH)")
plt.title("d轴电感-id 曲线")
plt.grid(True)
plt.legend()

# 绘制q轴电感-q轴电流曲线
plt.subplot(3, 1, 3)
plt.plot(Laq_vs_iq[:, 0], Laq_vs_iq[:, 1] * 1e3, marker='^', color='green', label='Lq-iq')
plt.xlabel("iq (A)")
plt.ylabel("Lq (mH)")
plt.title("q轴电感-iq 曲线")
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

## 2 计算函数定义

### 2.1 通用方法

#### 2.1.1 “加强版”interp函数，先排序再插值

In [ ]:
def interp_sorted(x_new, x_data, y_data, smooth=IsSmoothInterp):
	"""
	对数据按 x 排序后再插值（支持线性插值和光滑插值）
	
	参数:
		x_new: 要插值的新 x 值
		x_data: 原始 x 数据（可以是无序的）
		y_data: 原始 y 数据
		smooth: 是否使用光滑插值（三次样条）。True=光滑插值，False=线性插值
	
	返回:
		插值结果。当 y_data 为一维数组时返回标量值，多维数组时返回数组
	"""
	from scipy.interpolate import CubicSpline
	
	# 按 x 排序
	sorted_indices = np.argsort(x_data)
	x_sorted = x_data[sorted_indices]
	y_sorted = y_data[sorted_indices]
	
	# 插值
	if smooth:
		# 使用三次样条插值（光滑插值）
		# bc_type='natural' 表示自然边界条件（二阶导数为0）
		cs = CubicSpline(x_sorted, y_sorted, bc_type='natural')
		result = cs(x_new)
	else:
		# 使用线性插值
		result = np.interp(x_new, x_sorted, y_sorted)
	
	# 当 y_data 为一维数组且返回单个值时，转换为标量
	if isinstance(result, np.ndarray) and result.ndim == 0:
		return float(result)
	return result

#### 2.1.2 指定id、iq，查表Lad、Laq、PsiR、Ld、Lq

In [ ]:
# Lad、Laq、PsiR查表函数（支持线性插值和三次样条插值）
from scipy.interpolate import CubicSpline, interp1d

# 创建两种插值函数：线性插值和三次样条插值
# 线性插值器
_Lad_interp_linear = interp1d(Lad_vs_id[:, 0], Lad_vs_id[:, 1], kind='linear', fill_value='extrapolate')  # type: ignore
_Laq_interp_linear = interp1d(Laq_vs_iq[:, 0], Laq_vs_iq[:, 1], kind='linear', fill_value='extrapolate')  # type: ignore
_PsiR_interp_linear = interp1d(PsiR_vs_id[:, 0], PsiR_vs_id[:, 1], kind='linear', fill_value='extrapolate')  # type: ignore

# 三次样条插值器（光滑插值）
_Lad_interp_cubic = CubicSpline(Lad_vs_id[:, 0], Lad_vs_id[:, 1], bc_type='natural')
_Laq_interp_cubic = CubicSpline(Laq_vs_iq[:, 0], Laq_vs_iq[:, 1], bc_type='natural')
_PsiR_interp_cubic = CubicSpline(PsiR_vs_id[:, 0], PsiR_vs_id[:, 1], bc_type='natural')

def calcLad(id: float, smooth: bool = IsSmoothInterp) -> float:
	"""
	计算d轴电枢反应电感（支持线性/光滑插值）
	
	参数:
		id: d轴电流 (A)
		smooth: 是否使用光滑插值。True=三次样条插值，False=线性插值（默认True）
	
	返回:
		Lad: d轴电枢反应电感 (H)
	"""
	if smooth:
		Lad = _Lad_interp_cubic(id)
	else:
		Lad = _Lad_interp_linear(id)
	return float(Lad)

def calcLaq(iq: float, smooth: bool = IsSmoothInterp) -> float:
	"""
	计算q轴电枢反应电感（支持线性/光滑插值）
	
	参数:
		iq: q轴电流 (A)
		smooth: 是否使用光滑插值。True=三次样条插值，False=线性插值（默认True）
	
	返回:
		Laq: q轴电枢反应电感 (H)
	"""
	if smooth:
		Laq = _Laq_interp_cubic(iq)
	else:
		Laq = _Laq_interp_linear(iq)
	return float(Laq)

def calcPsiR(id: float, smooth: bool = IsSmoothInterp) -> float:
	"""
	计算永磁体磁链（支持线性/光滑插值）
	
	参数:
		id: d轴电流 (A)
		smooth: 是否使用光滑插值。True=三次样条插值，False=线性插值（默认True）
	
	返回:
		PsiR: 永磁体磁链 (Wb)
	"""
	if smooth:
		PsiR = _PsiR_interp_cubic(id)
	else:
		PsiR = _PsiR_interp_linear(id)
	return float(PsiR)

# Ld = Lad + Lsigma
def calcLd(id: float, smooth: bool = IsSmoothInterp) -> float:
	"""
	计算d轴电感
	
	参数:
		id: d轴电流 (A)
		smooth: 是否使用光滑插值。True=三次样条插值，False=线性插值（默认True）
	
	返回:
		Ld: d轴电感 (H)
	"""
	Lad = calcLad(id, smooth=smooth)
	Ld = Lad + Lsigma
	return Ld

# Lq = Laq + Lsigma
def calcLq(iq: float, smooth: bool = IsSmoothInterp) -> float:
	"""
	计算q轴电感
	
	参数:
		iq: q轴电流 (A)
		smooth: 是否使用光滑插值。True=三次样条插值，False=线性插值（默认True）
	
	返回:
		Lq: q轴电感 (H)
	"""
	Laq = calcLaq(iq, smooth=smooth)
	Lq = Laq + Lsigma
	return Lq 

#### 2.1.3 自适应步长单侧搜索目标值

In [ ]:
# 加载函数：自适应步长单侧搜索法
def adaptiveStepSearch(f, bracket, divisions=100, startFrom='left', target=0, tol=1e-5, maxIter=100):
	"""
	自适应步长单侧搜索算法
	
	在指定闭区间内从一端开始单侧搜索（包含端点）。
	当搜索方向一致时保持步长，方向改变时自动缩小步长（0.1倍），实现快速收敛。
	
	参数:
		f: 目标函数
		bracket: 搜索区间 [a, b]（闭区间），如 [-100, 0]
		divisions: 初始分段数，用于计算初始步长 = (b-a)/divisions
		startFrom: 起始方向，只能选择:
			- 'left': 从左端点a开始向右单侧搜索（默认）
			- 'right': 从右端点b开始向左单侧搜索
		target: 目标值（默认求根，即f(x)=0）
		tol: 收敛容差（绝对误差）
		maxIter: 最大迭代次数
	
	返回:
		(x, fx, iterations, converged, error_msg): 
			- x: 找到的解
			- fx: 对应的函数值
			- iterations: 实际迭代次数
			- converged: 是否收敛
	"""
	# 解析搜索区间
	a, b = bracket
	if a >= b:
		# 区间错误，报错
		raise ValueError(f"区间错误: bracket[0] ({a}) 必须小于 bracket[1] ({b})")
	
	# 根据startFrom参数确定起始点和初始步长方向
	if startFrom == 'left':
		x = a  # 从左端点开始
		step = abs(b - a) / divisions  # 向右搜索，步长为正
	elif startFrom == 'right':
		x = b  # 从右端点开始
		step = -abs(b - a) / divisions  # 向左搜索，步长为负
	else:
		raise ValueError(f"startFrom参数错误: 只能为 'left' 或 'right'，得到 '{startFrom}'")
	
	lastError = None
	lastX = None
	
	for i in range(maxIter):
		fx = f(x)
		error = fx - target
		
		# 检查收敛
		if abs(error) < tol:
			return x, fx, i + 1, True
		
		# 检查是否越过目标值（误差符号改变）
		if lastError is not None and error * lastError < 0:
			# 越过了目标值，退回到上一步
			x = lastX
			# 缩小步长（自适应二分逼近）
			step *= 0.1
		
		# 保存当前状态
		lastX = x
		lastError = error
		
		# 更新位置
		x += step

	print(f"未能在最大迭代次数 {maxIter} 内收敛，最后一次误差为 {error}")
	
	return x, f(x), maxIter, False

In [ ]:
# 测试：自适应步长单侧搜索法
print("=" * 70)
print("自适应步长单侧搜索算法 - 测试示例")
print("=" * 70)

# 测试1：简单二次函数求根 - 单侧搜索
print("\n[测试1] 求解: f(x) = x^2 - 4 = 0 (真实解: x = ±2)")
def testFunc1(x):
	return x**2 - 4

bracket1 = [-10, 10]      # 搜索区间[-10, 10]
divisions1 = 100        # 初始分为100段

print("  测试1a: 从左端点(0)开始向右单侧搜索")
xOpt, fx, iters, converged = adaptiveStepSearch(
	f=testFunc1,
	bracket=bracket1,
	divisions=divisions1,
	startFrom='left',    # 从左端点开始
	target=0,
	tol=1e-5
)
print(f"    搜索区间: [{bracket1[0]}, {bracket1[1]}] (闭区间)")
print(f"    初始分段数: {divisions1} (初始步长 = {(bracket1[1]-bracket1[0])/divisions1:.4f})")
print(f"    起始点: x0 = {bracket1[0]} (左端点)")
print(f"    找到解: x = {xOpt:.6f}")
print(f"    函数值: f(x) = {fx:.8f}")
print(f"    迭代次数: {iters}")
print(f"    收敛状态: {'成功' if converged else '未收敛'}")

print("\n  测试1b: 从右端点(10)开始向左单侧搜索")
xOpt2, fx2, iters2, converged2 = adaptiveStepSearch(
	f=testFunc1,
	bracket=bracket1,
	divisions=divisions1,
	startFrom='right',   # 从右端点开始
	target=0,
	tol=1e-5
)
print(f"    起始点: x0 = {bracket1[1]} (右端点)")
print(f"    找到解: x = {xOpt2:.6f}")
print(f"    迭代次数: {iters2}")
print(f"    收敛状态: {'成功' if converged2 else '未收敛'}")

#### 2.1.4 自适应步长搜索局部极大值

In [ ]:
# 加载函数：自适应步长搜索局部极大值
def adaptiveStepSearchMaximum(f, bracket, divisions=1000, startFrom='right', tol=1e-5, maxIter=200):
	"""
	自适应步长搜索局部极大值
	
	在指定闭区间内从一端开始单侧搜索局部最大值（包含端点）。
	当函数值持续增大时保持步长，函数值开始下降时退回并自动缩小步长（0.1倍），实现快速收敛。
	
	参数:
		f: 目标函数（需要求极大值）
		bracket: 搜索区间 [a, b]（闭区间）
		divisions: 初始分段数，用于计算初始步长 = (b-a)/divisions
		startFrom: 起始方向，只能选择:
			- 'left': 从左端点a开始向右单侧搜索
			- 'right': 从右端点b开始向左单侧搜索（默认）
		tol: 收敛容差（相对搜索范围）
		maxIter: 最大迭代次数
	
	返回:
		(x, fx, iterations, converged): 
			- x: 找到的极值点
			- fx: 对应的函数值
			- iterations: 实际迭代次数
			- converged: 是否收敛
	"""
	# 解析搜索区间
	a, b = bracket
	if a >= b:
		raise ValueError(f"区间错误: bracket[0] ({a}) 必须小于 bracket[1] ({b})")
	
	# 根据startFrom参数确定起始点和初始步长方向
	if startFrom == 'left':
		x = a # 从左端点开始
		step = abs(b-a) / divisions # 向右搜索，步长为正
	elif startFrom == 'right':
		x = b  # 从右端点开始
		step = -abs(b - a) / divisions  # 向左搜索，步长为负
	else:
		raise ValueError(f"startFrom参数错误: 只能为 'left' 或 'right'，得到 '{startFrom}'")
	
	# 当前函数值
	lastFx = f(x)
	lastX = x
	
	for i in range(maxIter):
		# 检查步长收敛
		if abs(step) < tol*abs(b - a):
			return lastX, lastFx, i + 1, True
		
		# 尝试前进一步
		x_new = lastX + step
		
		# 边界检查
		if startFrom == 'left' and x_new > b:
			x_new = b
		elif startFrom == 'right' and x_new < a:
			x_new = a
		
		fx_new = f(x_new)
		
		# 判断是否越过极大值（函数值开始下降）
		if fx_new < lastFx:
			# 函数值下降，说明越过了极大值
			# 向后退一步，缩小步长并反向
			lastX = lastX - step
			step = step * 0.1
		else:
			# 函数值上升或持平，继续前进
			lastX = x_new
			lastFx = fx_new
			
			# 检查是否到达边界
			if (startFrom == 'left' and x >= b) or (startFrom == 'right' and x <= a):
				return lastX, lastFx, i + 1, True
	
	return lastX, lastFx, maxIter, False

#### 2.1.5 计算电压/电流限幅

In [ ]:
# 加载函数：计算电压/电流限幅
def calcVILimit(Udc: float, Imax: float, connectType: str, modulationType: str) -> tuple[float, float]:
	"""
	计算电压/电流限幅
	
	参数:
		Udc: 变频器母线电压
		Imax: 变频器限幅电流(线电流幅值)
		connectType: 绕组连接方式，取值 'Y' 或 'D'
		modulationType: 调制方式，取值 'SVPWM' 或 'SPWM'
	
	返回:
		(UPmax, IPmax): 最大相电压幅值和最大相电流幅值
	"""
	if modulationType == 'SVPWM':
		ULmax = Udc
	elif modulationType == 'SPWM':
		ULmax = Udc / 2.0
	else:
		raise ValueError(f"不支持的调制方式: {modulationType}")
		
	if connectType == 'Y':
		UPmax = ULmax / np.sqrt(3.0)
	elif connectType == 'D':
		UPmax = ULmax
	else:
		raise ValueError(f"不支持的连接方式: {connectType}")

	ILmax = Imax
	if connectType == 'Y':
		IPmax = ILmax
	elif connectType == 'D':
		IPmax = ILmax * np.sqrt(2.0)
		
	return UPmax, IPmax

In [ ]:
# 测试：计算电压/电流限幅
UPmax, IPmax = calcVILimit(Udc, Imax, connectType, modulationType)
print(f"\n变频器母线电压 Udc = {Udc} V")
print(f"变频器限幅电流 Imax = {Imax} A")
print(f"绕组连接方式: {connectType}")
print(f"调制方式: {modulationType}")
print(f"计算得到的最大相电压幅值 UPmax = {UPmax:.2f} V")
print(f"计算得到的最大相电流幅值 IPmax = {IPmax:.2f} A")

### 2.2 电机性能计算方法

#### 2.2.1 给定id、iq，计算Tem

In [ ]:
# 加载函数：计算电磁转矩
def calcTem(id: float, iq: float) -> float:
	"""
	计算电磁转矩
	
	输入参数:
		id: d轴电流
		iq: q轴电流
	
	依赖的全局变量：
		polePairs: 极对数

	返回:
		Tem: 电磁转矩 (N·m)
	"""
	Lad = calcLad(id)
	Laq = calcLaq(iq)
	PsiR = calcPsiR(id)
	Tem = 1.5 * polePairs * (PsiR * iq + (Lad - Laq) * id * iq)
	return Tem

# 测试函数
Tem = calcTem(id=-50, iq=100)
print(f"电磁转矩: {Tem:.2f} N·m")


#### 2.2.2 给定id、iq、We，计算Up
- 定子电压幅值
	Us² = Ud² + Uq²
	= (ωe·Lq·iq - Rs·id)² + (ωe·(φf + Ld·id) + Rs·iq)²

In [ ]:
# 加载函数：给定dq电流和角速度，计算相电压幅值
def calcUp(id: float, iq: float, we: float) -> float:
	"""
	计算PMSM的定子相电压幅值

	输入参数:
		id: d轴电流
		iq: q轴电流
		we: 电角速度(rad/s)

	依赖的全局变量:
		Rs: 定子相电阻
		polePairs: 极对数

	返回:
		Up: 定子电压幅值
	"""
	# 计算d、q轴电感和磁链
	Ld = calcLad(id)
	Lq = calcLaq(iq)
	phi_f = calcPsiR(id)

	# Ud和Uq
	Ud = -we * Lq * iq + Rs * id
	Uq = we * (phi_f + Ld * id) + Rs * iq

	# 相电压幅值
	Up = np.sqrt(Ud**2 + Uq**2)
	return Up

#### 2.2.3 给定id、iq、Us，计算We
- 计算过程
  一元二次方程推导
  1. PMSM的dq轴电压方程
	  - Ud = -ωe·Lq·iq + Rs·id
	  - Uq = ωe·(φf + Ld·id) + Rs·iq
  2. 定子电压幅值
	  Us² = Ud² + Uq²
	  = (ωe·Lq·iq - Rs·id)² + (ωe·(φf + Ld·id) + Rs·iq)²
	  = ωe²·[(Lq·iq)² + (φf + Ld·id)²] + 2·ωe·Rs·[iq·(φf + Ld·id) - id·Lq·iq] + (Rs·iq)² + (Rs·id)²
  3. 整理为一元二次方程
	  [(Lq·iq)² + (φf + Ld·id)²]·ωe² + [2·Rs·iq·(φf + Ld·id - id·Lq)]·ωe + [(Rs·iq)² + (Rs·id)² - Us²] = 0
	  a = [(Lq·iq)² + (φf + Ld·id)²]
	  b = [2·Rs·iq·(φf + Ld·id - id·Lq)]
	  c = [(Rs·iq)² + (Rs·id)² - Us²]
  4. 判断方程解
	  - 判别式
		 delta = b² - 4·a·c
	  - 解方程
		 We = (-b + np.sqrt(delta)) / (2·a)
	  - 返回结果
		 return True, "", We

In [ ]:
# 加载函数：给定dq电流和相电压幅值，计算电角速度
def calcWe(id: float, iq: float, UP: float) -> tuple[bool, str, float]:
	"""
	给定dq电流和相电压幅值，计算电角速度
	
	参数:
		id: d轴电流
		iq: q轴电流
		UP: 相电压幅值
	
	依赖的全局变量：
		Rs: 定子相电阻

	返回:
		(success, error_msg, We): 是否成功、错误信息、电角速度
		
	原理：
		PMSM的dq轴电压方程：
			Ud = -ωe·Lq·iq + Rs·id
			Uq = ωe·(φf + Ld·id) + Rs·iq
		定子电压幅值：Us² = Ud² + Uq²
		整理为一元二次方程：
			[(Lq·iq)² + (φf + Ld·id)²]·ωe² + [2·Rs·iq·(φf + Ld·id - id·Lq)]·ωe + [(Rs·iq)² + (Rs·id)² - Us²] = 0
	"""
	Ld = calcLd(id)
	Lq = calcLq(iq)
	PsiR = calcPsiR(id)
	# 标准形式 a·ωe² + b·ωe + c = 0
	a = (Lq * iq)**2 + (PsiR + Ld * id)**2
	b = 2 * Rs * iq * (PsiR + Ld * id - id * Lq)
	c = (Rs * iq)**2 + (Rs * id)**2 - UP**2

	# 判别式
	delta = b**2 - 4*a*c

	if delta < 0:
		return False, "方程无实数解", 0.0
	else:
		We = (-b + np.sqrt(delta)) / (2*a)
		return True, "", We

#### 2.2.4 校核并计算去磁电流
- 校核id=-IPmax时，是否发生完全退磁（PsiR + Ldid < 0）
	- 若可以发生完全退磁，则计算去磁电流
	- 若不发生完全退磁，则函数直接返回 
- 计算去磁电流方法：
	- 本质是求解PsiR + Ldid = 0的解，其中PsiR和Ld是关于id的函数
	- 使用自适应步长的单侧搜索法求解（确保找到的abs(id)最小的解）
	- 搜索区间：
		- 初始步长：IPmax / deMagidPoints
		- 搜索区间：[-IPmax, 0]

In [ ]:
# 加载函数：校核并计算去磁电流
def checkAndCalcDemagid(IPmax: float, deMagidPoints: int) -> tuple[bool, float]:
	"""
	校核并计算去磁电流

	参数:
		IPmax: 最大电流幅值
		deMagidPoints: 去磁电流迭代初始等分数
	返回:
		(isDemag, id_demag): 是否发生完全退磁，去磁电流
	"""
	# 校核id = -IPmax时，是否发生完全退磁
	PsiR = calcPsiR(-IPmax)
	Lad = calcLad(-IPmax)
	if PsiR + Lad * (-IPmax) < 0:
		# 发生完全退磁，计算去磁电流
		def PsiD_pu(id):
			Lad = calcLad(id)
			PsiR = calcPsiR(id)
			PsiD = Lad * id + PsiR
			PsiD_pu = PsiD / PsiR
			return PsiD_pu
		# 使用自适应步长单侧搜索法求解(PsiR + Lad * id) / PsiR = 0
		id_demag, _, _, _ = adaptiveStepSearch(PsiD_pu, [-IPmax, 0], divisions=deMagidPoints, startFrom='right', target=0, tol=1e-5, )
		return True, id_demag
	else:
		# 不发生完全退磁，直接返回False
		return False, 0.0

In [ ]:
# 测试去磁电流计算函数
isDemag, id_demag = checkAndCalcDemagid(IPmax, deMagidPoints)
if isDemag:
	print(f"发生完全退磁，去磁电流为：{id_demag:.2f} A")
else:
	print("未发生完全退磁")

#### 2.2.5 给定iq、We、Us，计算id

**1. 定子电压方程**
	- Us² = Ud² + Uq²
	= (ωe·Lq·iq - Rs·id)² + (ωe·(φf + Ld·id) + Rs·iq)²

**2. 整理并按id配方系数收集，得标准二次方程**

a·id² + b·id + c = 0

其中：
- a = Rs² + ωe²·Ld²
- b = 2·ωe·[Ld·(ωe·φf + Rs·iq) - Rs·Lq·iq]
- c = (ωe·Lq·iq)² + (ωe·φf + Rs·iq)² - Us²

因此：

**id = (-b - √(b² - 4·a·c)) / (2·a)**
*（取左侧根）*

---

**推导：**

Us² = (ωe·Lq·iq - Rs·id)² + (ωe·(φf + Ld·id) + Rs·iq)²

展开第一项：
= (ωe·Lq·iq)² - 2·ωe·Lq·iq·Rs·id + Rs²·id²

展开第二项：+ (ωe·φf + Rs·iq + ωe·Ld·id)²

合并整理：
= (ωe·Lq·iq)² + (ωe·φf + Rs·iq)² + （2·ωe·[Ld·(ωe·φf + Rs·iq) - Rs·Lq·iq]）·id + (Rs² + ωe²·Ld²)·id²

移项并按id次数收集：
把 Us² 移到左侧并按 id 次数收集即可得到上面的 a、b、c。

In [ ]:
# 给定q轴电流、电角速度、相电压幅值，迭代计算d轴电流（可选左/右根）
def calcid(
	iq: float,
	We: float,
	Us: float,
	id_init: float = -100.0,
	maxIter: int = 50,
	tol: float = 1e-5,
	root: str = 'L'  # 'L'表示取左根（较小的/负根），'R'表示取右根（较大的/正根）
) -> tuple[bool, str, float]:
	"""
	给定q轴电流、电角速度、相电压幅值，计算d轴电流
	由于Ld和PsiR都是id的函数，需要迭代求解：
		a·id² + b·id + c = 0
	a = Rs² + We²·Ld²
	b = 2·We·[Ld·(We·PsiR + Rs·iq) - Rs·Lq·iq]
	c = (We·Lq·iq)² + (We·PsiR + Rs·iq)² - Us²

	参数:
		iq: q轴电流
		We: 电角速度
		Us: 相电压幅值
		id_init: d轴电流迭代初始值
		maxIter: 最大迭代次数
		tol: 收敛容差
		root: 'L'取左根（较小的/通常为负），'R'取右根（较大/通常为正）

	返回:
		(success, error_msg, id): 是否成功，错误信息，d轴电流
	"""
	id = id_init  # 初始化d轴电流
	Lq = calcLq(iq)

	for i in range(maxIter):
		# 查表获取Ld和PsiR
		Ld = calcLd(id)
		PsiR = calcPsiR(id)

		# 计算系数
		a = Rs**2 + We**2 * Ld**2
		b = 2 * We * (Ld * (We * PsiR + Rs * iq) - Rs * Lq * iq)
		c = (We * Lq * iq)**2 + (We * PsiR + Rs * iq)**2 - Us**2

		delta = b**2 - 4 * a * c
		if delta < 0:
			return False, "delta<0", 0.0

		sqrt_delta = np.sqrt(delta)
		if root.upper() == 'L':
			id_new = (-b - sqrt_delta) / (2 * a)
		elif root.upper() == 'R':
			id_new = (-b + sqrt_delta) / (2 * a)
		else:
			return False, "Unknown root param, use 'L' or 'R'", 0.0

		# 检查收敛
		if abs(id_new - id) / (abs(id) + 1e-8) < tol:
			return True, "", id_new

		# 更新id（阻尼系数）
		damping = 0.5  # 阻尼系数
		id = id + damping * (id_new - id)

	# 达到最大迭代次数仍未收敛
	return False, "maxIter", id

print("calcid函数定义完成！")

In [ ]:
# 测试calcid_L函数
print("=" * 70)
print("calcid函数测试")
print("=" * 70)

# 测试参数
iq_test = 50  # q轴电流
We_test = 400.0  # 电角速度 (rad/s)
Us_test = 310.0  # 相电压幅值 (V)
Lq_test = calcLq(iq_test)
id_init_test = -200  # 初始d轴电流

print(f"\n输入参数：")
print(f"  iq = {iq_test:.2f} A")
print(f"  We = {We_test:.2f} rad/s")
print(f"  Us = {Us_test:.2f} V")
print(f"  Lq = {Lq_test*1e3:.4f} mH")
print(f"  Rs = {Rs:.4f} Ω")
print(f"  id_init = {id_init_test:.2f} A")

# 调用函数
success, error_msg, idL_result = calcid(
	iq=iq_test,
	We=We_test,
	Us=Us_test,
	id_init=id_init_test,
	maxIter=100,
	tol=1e-5,
	root='L'
)
success, error_msg, idR_result = calcid(
	iq=iq_test,
	We=We_test,
	Us=Us_test,
	id_init=id_init_test,
	maxIter=100,
	tol=1e-5,
	root='R'
)

if success:
	print(f"\n✓ 计算成功！")
	print(f"  id 左根 = {idL_result:.4f}，右根 = {idR_result:.4f} A")
	
	for tag, id_result in zip(["左根", "右根"], [idL_result, idR_result]):
		print(f"\n--- 验证{tag} ---")
		Ld_result = calcLd(id_result)
		PsiR_result = calcPsiR(id_result)
		
		# 计算实际电压
		Ud = -(We_test * Lq_test * iq_test) + Rs * id_result
		Uq = We_test * (PsiR_result + Ld_result * id_result) + Rs * iq_test
		Us_calc = np.sqrt(Ud**2 + Uq**2)
		
		print(f"  Ld = {Ld_result*1e3:.4f} mH")
		print(f"  PsiR = {PsiR_result:.6f} Wb")
		print(f"  Ud = {Ud:.4f} V")
		print(f"  Uq = {Uq:.4f} V")
		print(f"  Us计算值 = {Us_calc:.4f} V")
		print(f"  Us目标值 = {Us_test:.4f} V")
		print(f"  误差 = {abs(Us_calc - Us_test):.6f} V ({abs(Us_calc - Us_test)/Us_test*100:.4f}%)")
		
		# 计算转矩
		Tem_result = calcTem(id_result, iq_test)
		print(f"  电磁转矩 = {Tem_result:.2f} N·m")
else:
	print(f"\n✗ 计算失败：{error_msg}")

#### 2.2.6 给定id、We、Us，计算iq

**1. 定子电压方程**
	- Us² = Ud² + Uq²
	= (ωe·Lq·iq - Rs·id)² + (ωe·(φf + Ld·id) + Rs·iq)²

**2. 整理并按iq配方系数收集，得标准二次方程**

a·iq² + b·iq + c = 0

其中：
- a = ωe²·Lq² + Rs²
- b = 2·ωe·Rs·[φf + (Ld - Lq)·id]
- c = (Rs·id)² + (ωe·(φf + Ld·id))² - Us²

因此：

**iq = (-b + √(b² - 4·a·c)) / (2·a)**

---

**推导过程（展开与合并）：**

Us² = (ωe·Lq·iq - Rs·id)² + (ωe·(φf + Ld·id) + Rs·iq)²

**展开第一项：**
(ωe·Lq·iq - Rs·id)² 
= (ωe·Lq)²·iq² - 2·ωe·Lq·Rs·id·iq + (Rs·id)²

**展开第二项：**
(ωe·(φf + Ld·id) + Rs·iq)² 
= (ωe·(φf + Ld·id))² + 2·ωe·(φf + Ld·id)·Rs·iq + Rs²·iq²

**合并所有项：**
Us² = (ωe·Lq)²·iq² + Rs²·iq²
	- 2·ωe·Lq·Rs·id·iq + 2·ωe·(φf + Ld·id)·Rs·iq
	+ (Rs·id)² + (ωe·(φf + Ld·id))²

**移项并按iq次数收集：**
0 = [(ωe·Lq)² + Rs²]·iq² + [2·ωe·Rs·(φf + Ld·id) - 2·ωe·Lq·Rs·id]·iq + [(Rs·id)² + (ωe·(φf + Ld·id))² - Us²]

**说明：**
- 当 Δ = b² - 4·a·c ≥ 0 时方程有实数解
- 通常有两个解，需要根据实际情况取正根
- 判别式 < 0 时，说明在给定的id和ωe下，无法满足电压约束Us

In [ ]:
# 加载函数：给定d轴电流、电角速度、相电压幅值，计算q轴电流
def calciq(id: float, We: float, Us: float, iq_init: float = 0, maxIter: int = 50, tol: float = 1e-6) -> tuple[bool, str, float]:
	"""
	给定d轴电流、电角速度、相电压幅值，计算q轴电流
	
	求解方程：a·iq² + b·iq + c = 0
	其中：
		a = We²·Lq² + Rs²
		b = 2·We·Rs·[PsiR + (Ld - Lq)·id]
		c = Rs²·id² + We²·(PsiR + Ld·id)² - Us²
	
	参数:
		id: d轴电流
		We: 电角速度
		Us: 相电压幅值
		iq_init: q轴电流迭代初始值
		maxIter: 最大迭代次数
		tol: 收敛容差

	依赖的全局变量：
		Rs: 定子相电阻

	返回:
		(success, error_msg, iq): 是否成功、错误信息、q轴电流
	"""
	iq = iq_init
	Ld = calcLd(id)
	PsiR = calcPsiR(id)

	for i in range(maxIter):
		# 根据当前iq查表获取Lq
		Lq = calcLq(iq)

		# 计算方程系数 a·iq² + b·iq + c = 0
		a = We**2 * Lq**2 + Rs**2
		b = 2 * We * Rs * (PsiR + (Ld - Lq) * id)
		c = Rs**2 * id**2 + We**2 * (PsiR + Ld * id)**2 - Us**2
	
		# 计算判别式
		delta = b**2 - 4 * a * c
	
		if delta < 0:
			return False, f"判别式为负（Δ={delta:.4e}），无实数解", 0.0
	
		# 求解二次方程
		iq_new = (-b + np.sqrt(delta)) / (2 * a)    

		# 检查收敛
		if abs(iq_new - iq)/iq < tol:
			return True, "", iq_new
		
		# 更新iq（使用阻尼系数提高收敛稳定性）
		damping = 0.5  # 阻尼系数
		iq = iq + damping * (iq_new - iq)
	
	# 达到最大迭代次数仍未收敛
	return False, "maxIter", iq

print("calciq函数定义完成！")

#### 2.2.7 给定Tem、iq，计算id
- Tem = 1.5 * polePairs * (PsiR * iq + (Ld - Lq) * id * iq)
- 给指定Tem_target、iq，给定的id_init作为右端点，使用adaptiveStepSearch()，计算Tem-Tem_target=0在允许误差范围内的id

In [ ]:
# 加载函数：指定Tem、iq，计算对应的id
def calcidByTemiq(Tem_target: float, iq: float, id_min: float, id_init: float = 0.0, divisions: int = 10000, maxIter: int = 1000) -> float:
	"""
	指定Tem、iq，计算对应的id
	
	Args:
		Tem_target: 目标电磁转矩 (N·m)
		iq: 电流iq (A)
		id_init: 初始id (A)
		id_min: 最小id (A)
	返回:
		id: 对应的id (A)
	"""
	id, _, iterations, converged = adaptiveStepSearch(
		f=lambda id: calcTem(id, iq) - Tem_target,
		bracket=[id_min, id_init],
		divisions=divisions,
		startFrom='right',
		target=0,
		tol=1e-5*Tem_target,
		maxIter=maxIter
	)
	return id

In [ ]:
# 测试calcidByTemiq函数
print("=" * 70)
print("calcidByTemiq函数测试")
print("=" * 70)

# 测试1：计算id_init=0时的id
Tem_target = 100.0
iq = 10
polePairs = 4
id_init = -IPmax
id = calcidByTemiq(Tem_target, iq, id_init, maxIter=10000)
print(f"Tem_target = {Tem_target:.2f} N·m, iq = {iq:.2f} A, id_init = {id_init:.2f} A, id = {id:.2f} A")

Tem_verify = calcTem(id, iq)
print(f"Tem_verify = {Tem_verify:.2f} N·m")

#### 2.2.8 给定Tem、id，计算iq

In [ ]:
# 加载函数：指定Tem、id，计算对应的iq
def calciqByTemid(Tem_target: float, id: float, iq_init: float, iq_min: float = 0.0, divisions: int = 10000, maxIter: int = 1000) -> float:
	"""
	指定Tem、id，计算对应的iq
	
	Args:
		Tem_target: 目标电磁转矩 (N·m)
		id: 电流id (A)
		iq_init: 初始iq (A)
		iq_min: 最小iq (A)
	返回:
		iq: 对应的iq (A)
	"""
	iq, _, iterations, converged = adaptiveStepSearch(
		f=lambda iq: calcTem(id, iq) - Tem_target,
		bracket=[iq_min, iq_init],
		divisions=divisions,
		startFrom='right',
		target=0,
		tol=1e-5*Tem_target,
		maxIter=maxIter
	)
	return iq

In [ ]:
# 测试calciqByTemid函数
print("=" * 70)
print("calciqByTemid函数测试")
print("=" * 70)

Tem_target = 10.0
id = -0.23
iq_init = 8.5
iq = calciqByTemid(Tem_target, id, iq_init, maxIter=10000)
print(f"Tem_target = {Tem_target:.2f} N·m, id = {id:.2f} A, iq_init = {iq_init:.2f} A, iq = {iq:.2f} A")

Tem_verify = calcTem(id, iq)
print(f"Tem_verify = {Tem_verify:.2f} N·m")

### 2.3 指定控制策略下的电流轨迹

#### 2.3.1 计算id=0电流轨迹
- 固定id=0，计算iq从0到IPmax，以id0Points等分，利用2.2.2和2.2.3计算对应的电磁转矩和电角速度
- 输出数据格式：[[iq0, id0, Is0, Tem0, maxWe0], [iq1, id1, Is1, Tem1, maxWe1], ...]

In [ ]:
# 加载函数：计算id=0电流轨迹
def calcid0Track(UPmax: float, IPmax: float, id0Points: int) -> np.ndarray:
	"""
	计算id=0时的电流轨迹
	
	原理：
	- 固定id=0，计算iq从0到IPmax，以id0Points等分
	- 对每个iq点：
	  - 计算电流幅值 Is = sqrt(id^2 + iq^2) = iq（因为id=0）
	  - 利用calcTem计算对应的电磁转矩 Tem = calcTem(id=0, iq)
	  - 利用calcWe计算对应的最大电角速度 maxWe = calcWe(id=0, iq, UPmax)
	
	参数:
		UPmax: 最大相电压幅值 (V)
		IPmax: 最大相电流幅值 (A)
		id0Points: 扫描点数
	
	返回:
		id0Track: id=0时的电流轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, maxWe0], [iq1, id1, Is1, Tem1, maxWe1], ...]
	"""
	
	print(f"\n{'='*70}")
	print(f"计算 id=0 电流轨迹")
	print(f"{'='*70}")
	print(f"参数：UPmax={UPmax:.2f} V, IPmax={IPmax:.2f} A, 扫描点数={id0Points}")
	
	id0Track = []
	
	# 从iq=0到iq=IPmax等分扫描
	for i in range(id0Points + 1):
		# 计算当前iq值
		iq = IPmax * i / id0Points
		
		# 固定id=0
		id = 0.0
		
		# 计算电流幅值 Is = sqrt(id^2 + iq^2) = iq
		Is = np.sqrt(id**2 + iq**2)
		
		# 计算电磁转矩
		Tem = calcTem(id=id, iq=iq)
		
		# 计算最大电角速度
		success, error_msg, maxWe = calcWe(id=id, iq=iq, UP=UPmax)
		
		if not success:
			print(f"  ✗ 第 {i+1}/{id0Points+1} 点计算失败: iq={iq:.2f} A, {error_msg}")
			maxWe = 0.0
		
		# 保存轨迹点 [iq, id, Is, Tem, maxWe]
		id0Track.append([iq, id, Is, Tem, maxWe])
	
	# 转换为numpy数组并返回
	id0Track = np.array(id0Track)
	
	return id0Track

In [ ]:
# 测试函数
print("\n" + "="*70)
print("测试 calcid0Track 函数")
print("="*70)

# 计算id=0轨迹
id0_track_test = calcid0Track(UPmax=UPmax, IPmax=IPmax, id0Points=50)

#### 2.3.2 计算MTPA电流轨迹
- **算法原理**：在给定电流幅值Is下，通过自适应步长搜索算法在id轴上寻找最大转矩点
	- 对每个电流幅值Is（从0到IPmax等分），定义目标函数 $T_{em}(I_d) = calcTem(I_d, \sqrt{I_s^2-I_d^2})$
	- 使用自适应步长搜索在区间 $[-I_s, I_{d,prev}]$ 内找到使 $T_{em}(I_d)$ 最大的 $I_d$
	- 利用轨迹连续性，后续搜索以前一点的结果作为搜索右端点，逐步缩小搜索范围
	- 计算对应的 $I_q = \sqrt{I_s^2-I_d^2}$，以及在电压限制下的最大转速 $\omega_{e,max} = calcWe(I_d, I_q, U_{Pmax})$
- **输出数据格式**：[[iq0, id0, Is0, Tem0, maxWe0], [iq1, id1, Is1, Tem1, maxWe1], ...]

In [ ]:
# 加载函数：计算MTPA电流轨迹（优化版，使用自适应步长搜索）
def calcIsTrackInMTPA(UPmax: float, IPmax: float, mtpaPoints: int, idDivisions=1000, idTol=1e-4, idMaxIter=200) -> np.ndarray:
	"""
	计算MTPA电流轨迹（优化版）
	
	使用自适应步长搜索算法在id轴上搜索最大转矩点，利用MTPA轨迹的连续性，后续搜索以上一次结果为参考点缩小搜索范围。
	
	参数:
		UPmax: 相电压幅值限幅
		IPmax: 最大电流幅值
		mtpaPoints: MTPA曲线扫描点数
		idDivisions: id搜索的初始分段数（默认1000）
		idTol: id搜索的收敛容差（默认1e-4）
		idMaxIter: id搜索的最大迭代次数（默认100）
	
	返回:
		mtpaTrack: MTPA电流轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, maxWe0], [iq1, id1, Is1, Tem1, maxWe1], ...]
	"""
	# 初始化结果列表
	mtpaTrack = []
	
	# 用于记录上一次的搜索结果
	last_id_max = None

	# 外层循环：遍历电流幅值
	for i in range(mtpaPoints + 1):
		I_temp = IPmax * i / mtpaPoints  # 计算当前电流幅值
		
		if I_temp < 1e-6:  # 电流接近0时，直接设置为0
			# 注意第一个点maxWe0暂时为空
			mtpaTrack.append([0.0, 0.0, 0.0, 0.0, calcWe(id=0.0, iq=0.0, UP=UPmax)[2]])
			last_id_max = 0.0  # 更新上次结果
			continue
		
		# 定义目标函数：给定id，计算转矩 Tem(id)
		def temFunc(id):
			# 根据电流约束计算iq
			iq = np.sqrt(I_temp**2 - id**2)
			# 计算转矩
			return calcTem(id, iq)
		
		# 确定搜索区间的右端点
		# 第一次搜索：右端点为0
		# 后续搜索：右端点为上次结果，利用轨迹连续性缩小搜索范围
		if last_id_max is None:
			bracket_right = 0.0
		else:
			bracket_right = last_id_max
		
		# 使用自适应步长搜索最大转矩点
		# 从id=last_id_max（上次的最大转矩位置）开始向左搜索到-I_temp
		id_max, T_max, iterations, converged = adaptiveStepSearchMaximum(
			f=temFunc,
			bracket=[-I_temp, bracket_right],  # id的搜索区间
			divisions=idDivisions,
			startFrom='right',
			tol=idTol,
			maxIter=idMaxIter
		)
		
		# 计算对应的iq
		iq_max = np.sqrt(I_temp**2 - id_max**2)

		# 计算当前电流点，在UPmax限制下的最大电角速度
		maxWe = calcWe(id=id_max, iq=iq_max, UP=UPmax)[2]
		
		# 保存MTPA轨迹点
		mtpaTrack.append([iq_max, id_max, I_temp, T_max, maxWe])
		
		# 更新上次的id结果，供下次迭代使用
		last_id_max = id_max
		
	# 转换为numpy数组并返回
	return np.array(mtpaTrack)


print("MTPA优化计算函数定义完成！")

In [ ]:
# MTPA电流轨迹计算-测试
print("开始计算MTPA轨迹...")
mtpaTrack = calcIsTrackInMTPA(UPmax=UPmax, IPmax=IPmax, mtpaPoints=100, idDivisions=1000)
print("MTPA轨迹计算完成！")
print(mtpaTrack)

# 提取结果
Is_Tem = mtpaTrack[:, [2, 3]]
id_Tem = mtpaTrack[:, [1, 3]]
iq_Tem = mtpaTrack[:, [0, 3]]
id_iq = mtpaTrack[:, [1, 0]]

print(f"\n计算得到 {len(id_iq)} 个MTPA轨迹点")
print(f"最大转矩: {Is_Tem[:, 1].max():.2f} N·m")


In [ ]:
# 可视化MTPA结果
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Ensure axes is always 2D
if not isinstance(axes, np.ndarray):
	axes = np.array([[axes]])
elif axes.ndim == 1:
	axes = axes.reshape(-1, 1)

# 1. id-iq轨迹
axes[0, 0].plot(id_iq[:, 0], id_iq[:, 1], 'b-o', markersize=3)
axes[0, 0].set_xlabel('id (A)')
axes[0, 0].set_ylabel('iq (A)')
axes[0, 0].set_title('MTPA电流轨迹 (id-iq平面)')
axes[0, 0].grid(True)
axes[0, 0].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[0, 0].axvline(x=0, color='k', linestyle='--', alpha=0.3)

# 2. 转矩-电流幅值曲线
axes[0, 1].plot(Is_Tem[:, 0], Is_Tem[:, 1], 'r-o', markersize=3)
axes[0, 1].set_xlabel('电流幅值 Is (A)')
axes[0, 1].set_ylabel('电磁转矩 Tem (N·m)')
axes[0, 1].set_title('转矩-电流幅值曲线')
axes[0, 1].grid(True)

# 3. 转矩-id曲线
axes[1, 0].plot(id_Tem[:, 0], id_Tem[:, 1], 'g-o', markersize=3)
axes[1, 0].set_xlabel('id (A)')
axes[1, 0].set_ylabel('电磁转矩 Tem (N·m)')
axes[1, 0].set_title('转矩-d轴电流曲线')
axes[1, 0].grid(True)

# 4. 转矩-iq曲线
axes[1, 1].plot(iq_Tem[:, 0], iq_Tem[:, 1], 'm-o', markersize=3)
axes[1, 1].set_xlabel('iq (A)')
axes[1, 1].set_ylabel('电磁转矩 Tem (N·m)')
axes[1, 1].set_title('转矩-q轴电流曲线')
axes[1, 1].grid(True)

plt.tight_layout()
plt.show()

#### 2.3.3 计算MTPV电流轨迹

**目标：** 在电压约束(UPmax)和电流约束(IPmax)下，计算不同转速We对应的最大转矩点，得到MTPV电流轨迹。

**计算流程：**

**步骤1：确定转速扫描范围**
- 计算最大扫描转速：`We_max = We_base × mtpvWeScanRatio`
- 扫描方向：从 `We_max` 开始，逐步降低 `We`

**步骤2：对每个转速点，搜索最大转矩对应的id和iq**

*2.1 定义转矩计算函数*
- 内部函数 `TemWithidAtMaxUs(id, We, iq_init)`：
	- 输入：d轴电流id、电角速度We、q轴电流初始值iq_init
	- 先调用 `calciq()` 计算在最大电压UPmax约束下的iq
	- 再根据id和iq计算电磁转矩Tem
	- 返回：电磁转矩Tem（作为id的函数）

*2.2 搜索最大转矩点*
- 使用 `adaptiveStepSearchMaximum()` 函数，单侧搜索 `TemWithidAtMaxUs(id)` 的最大值
- 初始值设定：
	- **第一个转速点** (We = We_max)：
		- id初始值：`id_init = id_demag`（去磁电流）
		- iq初始值：`iq_init = 0`
	- **后续转速点**：
		- id初始值：使用上一个转速点搜索得到的id值
		- iq初始值：使用上一个转速点搜索得到的iq值
- 搜索输出：该转速下最优id和iq

**步骤3：电流约束检查**
- 计算定子电流幅值：`Is = sqrt(id² + iq²)`
- 若 `Is > IPmax`，停止扫描（超出电流约束）
- 否则，继续降低We，进入下一个转速点的计算

**步骤4：返回结果**
- 输出：MTPV电流轨迹数组 `[iq, id, Is, Tem, We]`

In [ ]:
# 加载函数：计算MTPV电流轨迹
def calcMTPVIsTrack(We_base: float, mtpvWeScanRatio: float, UPmax: float, IPmax: float, 
					 id_demag: float, mtpvWePoints: int,
					 idDivisions=1000, idTol=1e-4, idMaxIter=100) -> np.ndarray:
	"""
	计算MTPV(最大转矩/功率比)电流轨迹
	
	【计算原理】
	MTPV轨迹是弱磁区域的最优电流轨迹。在电压约束(UPmax)和电流约束(IPmax)下，
	对于每个转速，在满足电压约束的等电压椭圆上找到最大转矩点，这些点连接起来
	就形成MTPV轨迹。
	
	【计算流程】
	步骤1：确定转速扫描范围 [We_base, We_max]，从高速向低速扫描
	步骤2：对每个转速点，通过内部函数和自适应搜索找到最大转矩点
		2.1 定义内部函数 TemWithidAtMaxUs(id) 计算给定id下的转矩
		2.2 使用 adaptiveStepSearchMaximum() 搜索最大转矩点
	步骤3：检查电流约束，若Is > IPmax则停止扫描
	步骤4：返回MTPV轨迹数组
	
	参数:
		We_base: 基速 (rad/s)，恒转矩区与弱磁区的分界转速
		mtpvWeScanRatio: MTPV转速扫描倍数(相对基速)，决定最大扫描转速
		UPmax: 最大相电压幅值 (V)
		IPmax: 最大相电流幅值 (A)
		id_demag: 去磁电流 (A)，用作第一个转速点的id初始值
		mtpvWePoints: 转速扫描点数
	
	依赖的全局变量：
		Rs: 定子相电阻 (Ω)
		polePairs: 极对数
	
	依赖的函数：
		calciq(): 计算给定id、We、Us下的iq
		calcTem(): 计算电磁转矩
		adaptiveStepSearchMaximum(): 自适应步长搜索最大值
	
	返回:
		mtpvTrack: MTPV电流轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, We0], ...]
				   轨迹顺序：从弱磁I/II区切换点(Is=IPmax) → 最高速点
	"""
	# ========== 步骤0：检查IPmax>id_demag ==========
	if IPmax <= abs(id_demag):
		raise ValueError(f"最大电流约束 IPmax={IPmax:.2f} A 必须大于去磁电流 |id_demag|={abs(id_demag):.2f} A")
	
	# ========== 步骤1：确定转速扫描范围 ==========
	# 计算最大扫描转速
	We_max = We_base * mtpvWeScanRatio
	
	# 初始化MTPV轨迹列表
	mtpvTrack = []
	
	# 初始化搜索初始值
	id_init = id_demag  # 第一个转速点使用去磁电流
	iq_init = 0.0       # 第一个转速点iq初始值为0
	
	# ========== 步骤2：对每个转速点搜索最大转矩对应的id和iq ==========
	# 从最大转速开始向下扫描到基速
	for i in range(mtpvWePoints + 1):
		# 计算当前转速（从We_max逐渐降低到We_base）
		We_current = We_max - (We_max - We_base) * i / mtpvWePoints
		
		# ------ 2.1 定义内部函数：计算给定id下在最大电压约束时的转矩 ------
		def TemWithidAtMaxUs(id: float) -> float:
			"""
			计算给定id在最大电压UPmax约束下的电磁转矩
			
			该函数对输入的id，调用calciq()计算在UPmax约束下的iq，
			然后计算并返回电磁转矩Tem。
			
			参数:
				id: d轴电流 (A)
			
			返回:
				Tem: 电磁转矩 (N·m)，如果计算失败返回-inf
			"""
			nonlocal iq_init  # 使用外层的iq_init作为计算初始值
			
			# 调用calciq()计算在UPmax约束下的iq
			iq_result = calciq(id=id, We=We_current, Us=UPmax, iq_init=iq_init)[2]
			
			# 检查calciq是否返回有效结果
			if iq_result is None or np.isnan(iq_result):
				return -np.inf  # 计算失败，返回负无穷
			
			# 更新iq_init为本次计算结果，作为下次的初始值
			iq_init = iq_result
			
			# 计算并返回电磁转矩
			Tem = calcTem(id=id, iq=iq_result)
			return Tem
		
		# ------ 2.2 使用自适应搜索找到最大转矩点 ------
		# 计算当前We下的最小id(iq=0)
		id_LowBound = calcid(iq=0, We=We_current, Us=UPmax, id_init=id_demag, root='L')[2]
		# 使用adaptiveStepSearchMaximum()单侧搜索TemWithidAtMaxUs()的最大值
		id_opt, Tem_max, maxIter, success = adaptiveStepSearchMaximum(
			f=TemWithidAtMaxUs,
			bracket=[id_LowBound,id_init],
			divisions=idDivisions,
			maxIter=idMaxIter,
			tol=idTol,
			startFrom='right'
		)
		
		# 获取最优id对应的iq（需要重新计算以确保准确）
		iq_opt = calciq(id=id_opt, We=We_current, Us=UPmax, iq_init=iq_init)[2]
		
		# 检查搜索是否成功
		if iq_opt is None or np.isnan(iq_opt) or Tem_max == -np.inf:
			# 搜索失败，跳过当前转速点
			continue
		
		# ========== 步骤3：电流约束检查 ==========
		# 计算最大转矩点的定子电流幅值
		Is_opt = np.sqrt(id_opt**2 + iq_opt**2)
		
		# 检查是否超出电流约束
		if Is_opt > IPmax:
			# 电流超限，停止扫描（已进入电流圆约束区域）
			break
		
		# 保存该转速下的MTPV点
		mtpvTrack.append([iq_opt, id_opt, Is_opt, Tem_max, We_current])
		
		# 更新下一次迭代的初始值
		# 后续转速点使用当前搜索得到的id和iq作为初始值
		id_init = id_opt
		iq_init = iq_opt
	
	# ========== 步骤4：返回结果 ==========
	# 转换为numpy数组
	mtpvTrack = np.array(mtpvTrack)
	
	# 反转顺序：使轨迹从弱磁I/II区切换点(低速) → 最高速点
	if len(mtpvTrack) > 0:
		mtpvTrack = np.flipud(mtpvTrack)
	
	return mtpvTrack

print("calcMTPVIsTrack函数定义完成！")

In [ ]:
# 测试calcMTPVIsTrack函数
print("=" * 70)
print("calcMTPVIsTrack函数测试")
print("=" * 70)

# 基速
We_base = 100.0

# 调用calcMTPVIsTrack函数
print(f"\n开始计算MTPV轨迹...")
mtpvTrack = calcMTPVIsTrack(
	We_base=We_base,
	UPmax=UPmax,
	IPmax=IPmax,
	id_demag=id_demag,
	mtpvWeScanRatio=100,  
	mtpvWePoints=5000 
)

print(f"\n✓ 计算完成！")
print(f"  MTPV轨迹点数: {len(mtpvTrack)}")
print(f"  最大转矩点（Is=IPmax处）:")
if len(mtpvTrack) > 0:
	iq_first, id_first, Is_first, Tem_first, We_first = mtpvTrack[0]
	print(f"    iq = {iq_first:.2f} A")
	print(f"    id = {id_first:.2f} A")
	print(f"    Tem = {Tem_first:.2f} N·m")
	print(f"    We_splitFW = {We_first:.2f} rad/s")
	# 绘制MTPV轨迹(id-iq平面)
	id_vs_iq = mtpvTrack[:, [1, 0]]
	plt.figure(figsize=(12, 5))
	plt.plot(id_vs_iq[:, 0], id_vs_iq[:, 1], 'bo-', label='MTPV轨迹 (id-iq)')
	plt.scatter(id_first, iq_first, color='r', marker='*', s=120, label='最大转矩点')
	plt.xlabel('id (A)')
	plt.ylabel('iq (A)')
	plt.title('MTPV轨迹在id-iq平面上的分布')
	plt.grid(True)
	plt.legend()
	plt.tight_layout()
	plt.show()

	# 绘制MTPV控制下的T-We曲线
	We_vs_Tem = mtpvTrack[:, [4, 3]]
	plt.figure(figsize=(12, 5))
	plt.plot(We_vs_Tem[:, 0], We_vs_Tem[:, 1], 'bo-', label='MTPV控制下的T-We曲线')
	plt.xlabel('We (rad/s)')
	plt.ylabel('Tem (N·m)')
	plt.title('MTPV控制下的T-We曲线')
	plt.grid(True)
	
else:
	print("    mtpvTrack为空，无法获取第一组数据")

#### 2.3.4 计算弱磁I区的电流轨迹
- 在MTPA电流轨迹中，找到Is=IPmax的点(即最后一个点)，作为弱磁I区的右端点(iq_right, id_right)
- 从MTPV电流轨迹中，找到Is=IPmax的点(即第一个点)，作为弱磁I区的左端点(iq_left, id_left)，如果未传入MTPV轨迹，则使用(-IPmax, 0)作为左端点
- iq从iq_left开始，逐步增加到iq_right，id = sqrt(IPmax^2 - iq^2)，得到弱磁I区的电流轨迹
- 利用calcTem计算对应的Tem
- 利用calcWe计算对应的We
- 返回弱磁I区的电流轨迹：fw1Track = [[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]   

In [ ]:
from typing import Optional

# 加载函数：计算弱磁I区的电流轨迹
def calcFW1Track(mtpaTrack: np.ndarray, UPmax: float, IPmax: float, fw1iqPoints: int, mtpvTrack: Optional[np.ndarray] = None) -> np.ndarray:
	"""
	计算弱磁I区的电流轨迹

	参数：
		mtpaTrack: MTPA电流轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]
		UPmax: 最大相电压幅值 (V)
		IPmax: 最大相电流幅值 (A)
		fw1iqPoints: 弱磁I区iq扫描点数
		mtpvTrack: 可选，MTPV电流轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, We0], ...]，默认 None
	
	依赖的全局变量：
		polePairs: 极对数
		Rs: 定子相电阻

	返回：
		fw1Track: 弱磁I区的电流轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]
	"""

	# 读取mtpaTrack中Is=IPmax的点
	iq_right = mtpaTrack[-1, 0]

	# 读取mtpvTrack中Is=IPmax的点，或使用默认值
	if mtpvTrack is not None:
		iq_left = mtpvTrack[0, 0]
	else:
		# 如果未传入MTPV轨迹，则使用 0.0 作为左端点（iq从右向左扫描）
		iq_left = 0.0

	# 计算弱磁I区的电流轨迹
	fw1Track = []
	for iq in np.linspace(iq_right, iq_left, fw1iqPoints):
		id = -np.sqrt(max(0.0, IPmax**2 - iq**2))
		Is = np.sqrt(id**2 + iq**2)
		Tem = calcTem(id, iq)
		We = calcWe(id, iq, UPmax)[2]
		fw1Track.append([iq, id, Is, Tem, We])

	return np.array(fw1Track)

In [ ]:
# 测试calcFW1Track函数
fw1Track = calcFW1Track(mtpaTrack=mtpaTrack, mtpvTrack=mtpvTrack, UPmax=UPmax, IPmax=IPmax, fw1iqPoints=fw1iqPoints)
print(f"fw1Track扫描点数: {len(fw1Track)}")

# 绘制id-iq轨迹
plt.figure(figsize=(12, 5))
plt.plot(fw1Track[:, 1], fw1Track[:, 0], 'bo-', label='弱磁I区电流轨迹 (id-iq)')
plt.xlabel('id (A)')
plt.ylabel('iq (A)')
plt.title('弱磁I区电流轨迹 (id-iq)')
plt.grid(True)

# 绘制We-Tem轨迹
plt.figure(figsize=(12, 5))
plt.plot(fw1Track[:, 4], fw1Track[:, 3], 'bo-', label='弱磁I区We-Tem轨迹')
plt.xlabel('We (rad/s)')
plt.ylabel('Tem (N·m)')
plt.title('弱磁I区We-Tem轨迹')
plt.grid(True)


#### 2.3.5 拼接可利用电流范围轨迹IsRange
- 输入：mtpaTrack或id0Track、fw1Track(可为空)、mtpvTrack(可为空)
- 将mtpaTrack或id0Track、fw1Track、mtpvTrack拼装成完整可用电流范围轨迹IsRange=[[iq0, id0], [iq1, id1], ...],并返回

In [ ]:
from typing import Optional

# 加载函数：拼装完整电流轨迹和We-Tem曲线
def calcFullTrack(mtpaTrack: np.ndarray, mtpvTrack: Optional[np.ndarray] = None, fw1Track: Optional[np.ndarray] = None) -> np.ndarray:
	"""
	计算完整的电流轨迹

	参数:
		mtpaTrack: MTPA轨迹 (N×4)，格式：[[iq0, id0, Is0, Tem0], [iq1, id1, Is1, Tem1], ...]
		mtpvTrack: MTPV轨迹 (N×5)，可选，格式：[[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]
		fw1Track: 弱磁I区轨迹 (N×5)，可选，格式：[[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]
	
	返回:
		IsRange: 完整的电流轨迹 (N×2)，格式：[[iq0, id0], [iq1, id1], ...]
	"""

	# 验证必需输入
	if mtpaTrack is None:
		raise ValueError("mtpaTrack must be provided and must be a numpy.ndarray")

	# 初始化IsRange
	IsRange = []

	# 拼接电流轨迹
	IsRange.extend(mtpaTrack[:, :2])
	if fw1Track is not None:
		IsRange.extend(fw1Track[:, :2])
	if mtpvTrack is not None:
		IsRange.extend(mtpvTrack[:, :2])

	return np.array(IsRange)

In [ ]:
# 测试calcFullTrack函数
IsRange = calcFullTrack(mtpaTrack=mtpaTrack, mtpvTrack=mtpvTrack, fw1Track=fw1Track)

# 绘制完整的电流轨迹和We-Tem曲线
plt.figure(figsize=(12, 5))
plt.plot(IsRange[:, 1], IsRange[:, 0], 'bo-', label='电流轨迹 (id-iq)')
plt.xlabel('id (A)')
plt.ylabel('iq (A)')
plt.title('可利用电流范围 (id-iq)')
plt.grid(True)

#### 2.3.6 计算等转矩线
- 在MTPA曲线插值获得对应的id_mtpa和iq_mtpa
- 判断左端点在弱磁I区/II区
	- 如果给定电磁转矩Tem>Tem_splitFW，或mtpvTrack为None，则左端点在弱磁I区
		- 对fw1Track进行插值获得对应的iq_fw1,id_fw1
		- (id_mtpa-id_fw1)>=(iq_mtpa-iq_fw1)
			- id作为扫描变量，则从id_mtpa开始，逐步减小id，步长(id_mtpa-id_fw1*0.95)/equTemPoints，利用calciqByTemid计算等转矩下的iq
		- (id_mtpa-id_fw1)<(iq_mtpa-iq_fw1)
			- iq作为扫描变量，则从iq_mtpa开始，逐步减小iq，步长(iq_mtpa-iq_fw1*0.95)/equTemPoints，利用calcidByTemiq计算等转矩下的id
	- 否则，左端点在弱磁II区
		- 获取mtpvTrack的最后一个点，得到minTemAtMtpv、minidAtMtpv、miniqAtMtpv
			- Tem_target<minTemAtMtpv
				- iq_mtpv=0.0,id_mtpv=minidAtMtpv
			- Tem_target>=minTemAtMtpv
				- 对mtpvTrack进行插值获得对应的iq_mtpv,id_mtpv
		- (id_mtpa-id_mtpv)>=(iq_mtpa-iq_mtpv)
			- id作为扫描变量，则从id_mtpa开始，逐步减小id，步长(id_mtpa-id_mtpv*0.95)/equTemPoints，利用calciqByTemid计算等转矩下的iq
		- (id_mtpa-id_mtpv)<(iq_mtpa-iq_mtpv)
			- iq作为扫描变量，则从iq_mtpa开始，逐步减小iq，步长(iq_mtpa-iq_mtpv*0.95)/equTemPoints，利用calcidByTemiq计算等转矩下的id
- 利用calcWe计算对应的We
- 返回等转矩轨迹：[[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]

In [ ]:
# type: ignore
# 加载函数：计算等转矩线
def calcIsTrackInEquTem(Tem_target: float, UPmax: float, IPmax: float, 
					mtpaTrack: np.ndarray, fw1Track: np.ndarray, mtpvTrack: Optional[np.ndarray] = None,
					equTemPoints: int = 50) -> np.ndarray:
	"""
	计算等转矩线

	参数:
		Tem_target: 目标电磁转矩 (N·m)
		UPmax: 最大相电压幅值 (V)
		IPmax: 最大相电流幅值 (A)
		mtpaTrack: MTPA轨迹 (N×4)，格式：[[iq0, id0, Is0, Tem0], [iq1, id1, Is1, Tem1], ...]
		fw1Track: 弱磁I区轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]
		mtpvTrack: MTPV轨迹 (N×5)，可选，格式：[[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]
		equTemPoints: 等转矩点数
	返回:
		equTemTrack: 等转矩轨迹，格式：[[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]
	"""

	# 获得fw1Track上的，最大电磁转矩Tem_max（fw1Track的第一个点）和最小电磁转矩Tem_splitFW（fw1Track的最后一个点）
	Tem_max = fw1Track[0, 3]
	Tem_splitFW = fw1Track[-1, 3]

	# 判断给定转矩是否超出最大电磁转矩
	if Tem_target > Tem_max:
		# 目标转矩大于最大转矩，报错
		raise ValueError(f"目标转矩{Tem_target:.2f} N·m大于最大转矩{Tem_max:.2f} N·m")
	
	# 在MTPA上查找对应的id_mtpa和iq_mtpa
	id_mtpa = interp_sorted(Tem_target, mtpaTrack[:, 3], mtpaTrack[:, 1])
	iq_mtpa = interp_sorted(Tem_target, mtpaTrack[:, 3], mtpaTrack[:, 0])
	print(f"iq_mtpa = {iq_mtpa:.2f} A, id_mtpa = {id_mtpa:.2f} A")

	lastPoint = []
	# 判断等转矩线的左端点在弱磁I区/II区
	# 如果给定电磁转矩Tem>Tem_splitFW，或mtpvTrack为None，则左端点在弱磁I区
	if Tem_target > Tem_splitFW or mtpvTrack is None:
		# 左端点在弱磁I区，对fw1Track进行插值获得对应的iq_fw1和id_fw1
		iq_fw1 = interp_sorted(Tem_target, fw1Track[:, 3], fw1Track[:, 0])
		id_fw1 = interp_sorted(Tem_target, fw1Track[:, 3], fw1Track[:, 1])
		Is_fw1 = interp_sorted(Tem_target, fw1Track[:, 3], fw1Track[:, 2])
		Tem_fw1 = interp_sorted(Tem_target, fw1Track[:, 3], fw1Track[:, 3])
		We_fw1 = interp_sorted(Tem_target, fw1Track[:, 3], fw1Track[:, 4])
		lastPoint = [iq_fw1, id_fw1, Is_fw1, Tem_fw1, We_fw1]
		print(f"左端点在弱磁I区: iq_fw1 = {iq_fw1:.2f} A, id_fw1 = {id_fw1:.2f} A")
		
		# 判断(id_mtpa-id_fw1)>=(iq_mtpa-iq_fw1)，决定扫描变量
		id_diff = abs(id_mtpa - id_fw1)
		iq_diff = abs(iq_mtpa - iq_fw1)
		
		if id_diff >= iq_diff:
			# id作为扫描变量，从id_mtpa开始，逐步减小id，利用calciqByTemid计算等转矩下的iq
			scan_by_id = True
			id_step = (id_mtpa - id_fw1 * 0.95) / equTemPoints
			print(f"选择id作为扫描变量, id_step = {id_step:.2f} A")
		else:
			# iq作为扫描变量，从iq_mtpa开始，逐步减小iq，利用calcidByTemiq计算等转矩下的id
			scan_by_id = False
			iq_step = (iq_mtpa - iq_fw1 * 0.95) / equTemPoints
			print(f"选择iq作为扫描变量, iq_step = {iq_step:.2f} A")
	else:
		# 左端点在弱磁II区
		# 获取mtpvTrack的最后一个点
		minTemAtMtpv = mtpvTrack[-1, 3]
		minidAtMtpv = mtpvTrack[-1, 1]
		
		if Tem_target < minTemAtMtpv:
			iq_mtpv = 0.0
			id_mtpv = minidAtMtpv
			print(f"左端点在弱磁II区(Tem<minTemAtMtpv): iq_mtpv = {iq_mtpv:.2f} A, id_mtpv = {id_mtpv:.2f} A")
		else:
			# 对mtpvTrack进行插值获得对应的iq_mtpv和id_mtpv
			iq_mtpv = interp_sorted(Tem_target, mtpvTrack[:, 3], mtpvTrack[:, 0])
			id_mtpv = interp_sorted(Tem_target, mtpvTrack[:, 3], mtpvTrack[:, 1])
			Is_mtpv = interp_sorted(Tem_target, mtpvTrack[:, 3], mtpvTrack[:, 2])
			Tem_mtpv = interp_sorted(Tem_target, mtpvTrack[:, 3], mtpvTrack[:, 3])
			We_mtpv = interp_sorted(Tem_target, mtpvTrack[:, 3], mtpvTrack[:, 4])
			lastPoint = [iq_mtpv, id_mtpv, Is_mtpv, Tem_mtpv, We_mtpv]
			print(f"左端点在弱磁II区: iq_mtpv = {iq_mtpv:.2f} A, id_mtpv = {id_mtpv:.2f} A")
		
		# 判断(id_mtpa-id_mtpv)>=(iq_mtpa-iq_mtpv)，决定扫描变量
		id_diff = abs(id_mtpa - id_mtpv)
		iq_diff = abs(iq_mtpa - iq_mtpv)
		
		if id_diff >= iq_diff:
			# id作为扫描变量，从id_mtpa开始，逐步减小id，利用calciqByTemid计算等转矩下的iq
			scan_by_id = True
			id_step = (id_mtpa - id_mtpv * 0.95) / equTemPoints
			print(f"选择id作为扫描变量, id_step = {id_step:.2f} A")
		else:
			# iq作为扫描变量，从iq_mtpa开始，逐步减小iq，利用calcidByTemiq计算等转矩下的id
			scan_by_id = False
			iq_step = (iq_mtpa - iq_mtpv * 0.95) / equTemPoints
			print(f"选择iq作为扫描变量, iq_step = {iq_step:.2f} A")
	
	# 初始化等转矩轨迹列表
	equTemTrack = []
	
	# 根据扫描变量进行迭代
	if scan_by_id:
		# id作为扫描变量，利用calciqByTemid计算等转矩下的iq
		iq_init = iq_mtpa*1.05   # 考虑MTPA电流轨迹的插值误差
		
		for i in range(equTemPoints + 1):
			# 计算当前id
			id = id_mtpa - i * id_step
			
			# 利用calciqByTemid计算等转矩下的iq
			iq = calciqByTemid(Tem_target, id, iq_init)
			if iq < 0.0:
				print(f'iq={iq},iq<0, 结束计算等转矩轨迹')
				break
			Tem = calcTem(id, iq)
			if abs(Tem - Tem_target)/Tem_target > 1e-2:
				print(f"Tem_target = {Tem_target:.2f} N·m, id = {id:.2f} A, iq_init = {iq_init:.2f} A")
				print(f"iq = {iq:.2f} A")
				print(f'计算得到的转矩Tem={Tem:.2f}与目标转矩Tem_target={Tem_target:.2f}误差过大')
			
			Is = np.sqrt(id**2 + iq**2)
			# 判断是否超出电流限制
			if Is > IPmax:
				print(f"电流超限: Is = {Is:.2f} A > IPmax = {IPmax:.2f} A，停止循环")
				break
			
			# 计算电角速度We
			success, error_msg, We = calcWe(id, iq, UPmax)
			# 判断We是否有解
			if not success:
				print(f"We无解: {error_msg}，停止循环")
				break
			
			# 保存当前点到等转矩轨迹
			equTemTrack.append([iq, id, Is, Tem, We])
			
			# 更新iq_init为当前iq，供下一次迭代使用
			iq_init = iq
	else:
		# iq作为扫描变量，利用calcidByTemiq计算等转矩下的id
		id_init = id_mtpa*1.05  # 考虑MTPA电流轨迹的插值误差
		
		for i in range(equTemPoints + 1):
			# 计算当前iq
			iq = iq_mtpa - i * iq_step
			
			# 利用calcidByTemiq计算等转矩下的id
			id = calcidByTemiq(Tem_target, iq, id_init, -IPmax)
			if id < IPmax:
				break
			Tem = calcTem(id, iq)
			if abs(Tem - Tem_target)/Tem_target > 1e-2:
				print(f"Tem_target = {Tem_target:.2f} N·m, iq = {iq:.2f} A, id_init = {id_init:.2f} A")
				print(f"id = {id:.2f} A")
				print(f'计算得到的转矩Tem={Tem:.2f}与目标转矩Tem_target={Tem_target:.2f}误差过大')
			
			Is = np.sqrt(id**2 + iq**2)
			# 判断是否超出电流限制
			if Is > IPmax:
				print(f"电流超限: Is = {Is:.2f} A > IPmax = {IPmax:.2f} A，停止循环")
				break
			
			# 计算电角速度We
			success, error_msg, We = calcWe(id, iq, UPmax)
			
			# 判断We是否有解
			if not success:
				print(f"We无解: {error_msg}，停止循环")
				break
			
			# 保存当前点到等转矩轨迹
			equTemTrack.append([iq, id, Is, Tem, We])
			
			# 更新id_init为当前id，供下一次迭代使用
			id_init = id
	
	if lastPoint is not None and len(lastPoint) > 0:
		# 将fw1Track或mtpvTrack与等转矩线的交点（插值获得）作为equTemTrack的最后一个点
		equTemTrack.append(lastPoint)

	# 转换为numpy数组并返回
	return np.array(equTemTrack)

print("calcIsTrackInEquTem函数定义完成！")

In [ ]:
# 测试calcIsTrackInTem函数
equTemTrack = calcIsTrackInEquTem(Tem_target=50.0, 
							  UPmax=UPmax, 
							  IPmax=IPmax, 
							  mtpaTrack=mtpaTrack, 
							  fw1Track=fw1Track, 
							  mtpvTrack=mtpvTrack,
							  equTemPoints=equTemPoints)

# 绘制等转矩线
plt.figure(figsize=(12, 5))
plt.plot(equTemTrack[:, 1], equTemTrack[:, 0], 'bo-', label='等转矩线 (id-iq)')
plt.xlabel('id (A)')
plt.ylabel('iq (A)')
plt.title('等转矩线 (id-iq)')
plt.grid(True)

#### 2.3.7 计算给定转速的等电压椭圆
**功能描述**：给定电角速度ωe、相电压幅值UPmax、MTPA轨迹、弱磁I区轨迹和MTPV轨迹，计算对应转速下电流控制范围内的等电压圆电流轨迹IsTrackInEquU。

**计算流程**：

1. **转速范围判断**
	- 若 ωe ≤ maxWeAtMtpaTrackEnd（MTPA轨迹末端对应的最大转速），则等电压椭圆在电流控制范围外，返回空值
	- 若 ωe > maxWeAtMtpaTrackEnd，继续步骤2

2. **确定轨迹起点(id_start, iq_start)**
	- 若 ωe ≤ maxWeAtMtpaTrackStart（MTPA轨迹起点对应的最大转速），则等电压椭圆与MTPA轨迹相交，在mtpaTrack中对ωe进行插值获得起点
	- 若 ωe > maxWeAtMtpaTrackStart，则等电压椭圆在MTPA轨迹外侧，使用下述方法确定起点：
	  - 计算当前电压圆与id轴的右交点：id_right = calcid(iq=0, ωe=ωe, Us=UPmax, id_init=0, root='R')
	  - 计算当前电压圆的最大iq值：iq_max = calciq(id=id_demag, ωe=ωe, Us=UPmax, iq_init=0)
	  - 设起点为 (id_start, iq_start) = (id_right, 0)

3. **确定轨迹终点(id_end, iq_end)**
	- 若 ωe ≤ maxWeAtFw1TrackStart（弱磁I区轨迹起点对应的最大转速），或mtpvTrack为None，则等电压椭圆与弱磁I区轨迹相交，在fw1Track中对ωe进行插值获得终点
	- 否则，等电压椭圆与MTPV轨迹相交，在mtpvTrack中对ωe进行插值获得终点

4. **构建等电压椭圆轨迹**
	- [iq_start,iq_max/splitRatio]等分，利用calcid(root='R')计算对应的id，得到等电压椭圆电流轨迹IsTrackInEquU,如果iq_start <= iq_max/splitRatio，则跳过这一步
	- 计算iq_max/splitRatio对应id_split，如果iq_start > iq_max/splitRatio，则将id_start作为id_split
	- 从(id_split,id_end)等分，利用calciq()计算对应的iq，补充完整等电压椭圆电流轨迹IsTrackInEquU
	- 综合两段点集构成完整的等电压椭圆电流轨迹IsTrackInEquU

In [ ]:
# type: ignore
# 加载函数：计算指定转速的等电压椭圆
def calcIsTrackInEquU(We: float, mtpaTrack: np.ndarray, fw1Track: np.ndarray,
					 UPmax: float, IPmax: float, id_demag: float,
					 mtpvTrack: Optional[np.ndarray] = None,
					 equUPoints: int = 100, splitRatio: int = 5) -> tuple[np.ndarray, float]:
	"""
	计算指定转速下电流控制范围内的等电压椭圆轨迹
	
	参数:
		We: 电角速度 (rad/s)
		mtpaTrack: MTPA轨迹 (N×4)，格式：[[iq0, id0, Is0, Tem0], ...]
		fw1Track: 弱磁I区轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, We0], ...]
		mtpvTrack: MTPV轨迹 (N×5)，可选，格式：[[iq0, id0, Is0, Tem0, We0], ...]
		UPmax: 最大相电压幅值 (V)
		IPmax: 最大相电流幅值 (A)
		id_demag: 去磁电流 (A)
		equUPoints: 等电压椭圆扫描点数
		splitRatio: 分段比例，用于iq从0到iq_max的分段
	
	返回:
		IsTrackInEquU: 等电压椭圆电流轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]
		Tem_max: 等电压椭圆对应的最大转矩
	"""
	
	# ========== 步骤1：转速范围判断 ==========
	# 获取MTPA轨迹末端对应的最大转速
	maxWeAtMtpaTrackEnd = 0
	if len(mtpaTrack) > 0:
		# 从MTPA轨迹的最后一个点，使用calcWe计算对应的最大转速
		id_end = mtpaTrack[-1, 1]
		iq_end = mtpaTrack[-1, 0]
		maxWeAtMtpaTrackEnd = calcWe(id_end, iq_end, UPmax)[2]
	
	# 获取MTPA轨迹起点对应的最大转速
	maxWeAtMtpaTrackStart = 0
	if len(mtpaTrack) > 0:
		id_start = mtpaTrack[0, 1]
		iq_start = mtpaTrack[0, 0]
		maxWeAtMtpaTrackStart = calcWe(id_start, iq_start, UPmax)[2]
	
	print(f"[步骤1] 转速范围判断")
	print(f"  输入转速: We = {We:.2f} rad/s")
	print(f"  MTPA末端最大转速: maxWeAtMtpaTrackEnd = {maxWeAtMtpaTrackEnd:.2f} rad/s")
	print(f"  MTPA起点最大转速: maxWeAtMtpaTrackStart = {maxWeAtMtpaTrackStart:.2f} rad/s")
	
	# 判断转速范围
	if We <= maxWeAtMtpaTrackEnd:
		# 等电压椭圆在电流控制范围外，返回空
		print(f"  ✗ We <= maxWeAtMtpaTrackEnd，转速过低，返回空轨迹")
		return np.array([]), 0.0
	else:
		print(f"  ✓ We > maxWeAtMtpaTrackEnd，转速在有效范围内")
	
	# ========== 步骤2：确定轨迹起点(id_start, iq_start) ==========
	print(f"\n[步骤2] 确定轨迹起点")
	if We <= maxWeAtMtpaTrackStart:
		# 等电压椭圆与MTPA轨迹相交，在mtpaTrack中对We进行插值获得起点
		print(f"  分支A: We <= maxWeAtMtpaTrackStart")
		print(f"    起点在MTPA轨迹内，使用插值方法")
		We_mtpa = mtpaTrack[:, 4] if mtpaTrack.shape[1] > 4 else None
		We_mtpa = mtpaTrack[:, 4] if mtpaTrack.shape[1] > 4 else None
		if We_mtpa is None:
			We_mtpa = np.array([mtpaTrack[i, 4] for i in range(len(mtpaTrack))])
		# We_mtpa是从大到小排序的，插值前，要先把它反转过来
		We_mtpa = We_mtpa[::-1]
		id_start = interp_sorted(We, We_mtpa, mtpaTrack[::-1, 1])
		iq_start = interp_sorted(We, We_mtpa, mtpaTrack[::-1, 0])
		print(f"    ✓ 插值得到起点: id_start = {id_start:.2f} A, iq_start = {iq_start:.2f} A")
	else:
		# 等电压椭圆在MTPA轨迹外侧
		# 计算当前电压圆与id轴的右交点
		print(f"  分支B: We > maxWeAtMtpaTrackStart")
		print(f"    起点在MTPA轨迹外侧")
		success_R, msg_R, id_right = calcid(iq=0, We=We, Us=UPmax, id_init=id_demag, root='R')
		
		if success_R:
			id_start = id_right
			print(f"    ✓ 计算电压圆与id轴右交点: id_start = {id_start:.2f} A")
		else:
			id_start = 0
			print(f"    ✗ 计算失败，取默认值: id_start = 0")
		
		# 计算当前电压圆的最大iq值
		success, msg, iq_max_temp = calciq(id=id_demag, We=We, Us=UPmax, iq_init=0)
		
		iq_start = 0
		print(f"    设置起点q轴电流: iq_start = 0")
	
	# ========== 步骤3：确定轨迹终点(id_end, iq_end) ==========
	print(f"\n[步骤3] 确定轨迹终点")
	# 获取fw1Track中第一个点对应的最大转速（弱磁I区起点对应的最大转速）
	maxWeAtFw1TrackStart = fw1Track[-1, 4] 

	print(f"  弱磁I区终点最大转速: maxWeAtFw1TrackStart = {maxWeAtFw1TrackStart:.2f} rad/s")
	
	if We <= maxWeAtFw1TrackStart or mtpvTrack is None:
		# 等电压椭圆与弱磁I区轨迹相交，在fw1Track中对We进行插值获得终点
		print(f"  分支A: We <= maxWeAtFw1TrackStart 或 mtpvTrack为None")
		print(f"    终点在弱磁I区轨迹内，使用插值方法")
		We_fw1 = fw1Track[:, 4] if fw1Track.shape[1] > 4 else None
		if We_fw1 is None:
			We_fw1 = np.array([fw1Track[i, 4] for i in range(len(fw1Track))])
		id_end = interp_sorted(We, We_fw1, fw1Track[:, 1])
		iq_end = interp_sorted(We, We_fw1, fw1Track[:, 0])
		print(f"    ✓ 插值得到终点: id_end = {id_end:.2f} A, iq_end = {iq_end:.2f} A (弱磁I区)")
	else:
		# 等电压椭圆与MTPV轨迹相交，在mtpvTrack中对We进行插值获得终点
		print(f"  分支B: We > maxWeAtFw1TrackStart 且 mtpvTrack不为None")
		print(f"    终点在MTPV轨迹内，使用插值方法")
		We_mtpv = mtpvTrack[:, 4] if mtpvTrack.shape[1] > 4 else None
		if We_mtpv is None:
			We_mtpv = np.array([mtpvTrack[i, 4] for i in range(len(mtpvTrack))])
		id_end = interp_sorted(We, We_mtpv, mtpvTrack[:, 1])
		iq_end = interp_sorted(We, We_mtpv, mtpvTrack[:, 0])
		print(f"    ✓ 插值得到终点: id_end = {id_end:.2f} A, iq_end = {iq_end:.2f} A (MTPV区)")
	
	# ========== 步骤4：构建等电压椭圆轨迹 ==========
	print(f"\n[步骤4] 构建等电压椭圆轨迹")
	# 计算当前电压圆的最大iq值
	success, msg, iq_max = calciq(id=id_demag, We=We, Us=UPmax, iq_init=0)
	
	if not success or iq_max is None or np.isnan(iq_max) or iq_max <= 0:
		print(f"  无法计算iq_max或其值无效，设置为0")
		iq_max = 0
	else:
		print(f"  等电压圆最大iq值: iq_max = {iq_max:.2f} A")
	
	IsTrackInEquU = []
	
	# 计算阈值
	iq_threshold = iq_max / splitRatio if splitRatio > 0 else 0
	print(f"  iq扫描阈值: iq_threshold = iq_max / {splitRatio} = {iq_threshold:.2f} A")
	print(f"  起点iq值: iq_start = {iq_start:.2f} A")
	
	# 第一段：[iq_start, iq_max/splitRatio]等分，利用calcid(root='R')计算对应的id
	# 条件：如果iq_start <= iq_threshold，则执行第一段
	if iq_start <= iq_threshold:
		print(f"  分支A: iq_start <= iq_threshold，执行第一段计算")
		print(f"    第一段范围: iq从 {iq_start:.2f} A 到 {iq_threshold:.2f} A")
		iq_points_1 = np.linspace(iq_start, iq_threshold, equUPoints // 2)
		
		count_seg1 = 0
		id_init_value = id_start  # 第一次使用id_start作为初值
		for iq in iq_points_1:
			success, msg, id = calcid(iq=iq, We=We, Us=UPmax, id_init=id_init_value, root='R')
			if success and id is not None and not np.isnan(id):
				IsTrackInEquU.append([iq, id, np.sqrt(iq**2 + id**2), calcTem(id, iq), We])
				id_init_value = id  # 之后使用上一步计算的id
				count_seg1 += 1
		print(f"    ✓ 第一段得到 {count_seg1} 个有效点")
		
		# 计算iq=iq_threshold对应的id_split
		id_split = IsTrackInEquU[-1][1]
		iq_split = IsTrackInEquU[-1][0]
		print(f"    计算得到 id_split = {id_split:.2f} A")

	else:
		# 条件：如果iq_start > iq_threshold，则跳过第一段，直接使用id_start作为id_split
		print(f"  分支B: iq_start > iq_threshold，跳过第一段")
		print(f"    直接将 id_split = id_start = {id_start:.2f} A")
		id_split = id_start
		iq_split = iq_start
	
	# 第二段：从(id_split, id_end)等分，利用calciq()计算对应的iq
	print(f"  第二段范围: id从 {id_split:.2f} A 到 {id_end:.2f} A")
	id_points_2 = np.linspace(id_split, id_end, equUPoints // 2)
	
	count_seg2 = 0
	iq_init_value = iq_split  # 第一次使用iq_split作为初值
	for id in id_points_2:
		success, msg, iq = calciq(id=id, We=We, Us=UPmax, iq_init=iq_init_value)
		if success and iq is not None and not np.isnan(iq):
			IsTrackInEquU.append([iq, id, np.sqrt(iq**2 + id**2), calcTem(id, iq), We])
			iq_init_value = iq  # 之后使用上一步计算的iq
			count_seg2 += 1
	print(f"  ✓ 第二段得到 {count_seg2} 个有效点")
	
	# 转换为numpy数组
	IsTrackInEquU = np.array(IsTrackInEquU)
	print(IsTrackInEquU)
	
	print(f"  ✓ 轨迹总点数（去重后）: {len(IsTrackInEquU)}")
	
	# 计算最大转矩
	Tem_max = 0.0
	for i in range(len(IsTrackInEquU)):
		iq_i = IsTrackInEquU[i, 0]
		id_i = IsTrackInEquU[i, 1]
		Tem_i = calcTem(id_i, iq_i)
		if Tem_i > Tem_max:
			Tem_max = Tem_i
	
	print(f"  ✓ 最大转矩: Tem_max = {Tem_max:.2f} N·m")
	print(f"\n[完成] calcIsTrackInEquU函数执行完成！")
	
	return IsTrackInEquU, Tem_max

print("calcIsTrackInEquU函数定义完成！")

In [ ]:
# 测试calcIsTrackInEquU函数
print("=" * 70)
print("calcIsTrackInEquU函数测试")
print("=" * 70)

# 测试参数
We_test = 300  # 电角速度 (rad/s)

print(f"\n输入参数：")
print(f"  We = {We_test:.2f} rad/s")
print(f"  n = {We_test * 9.55 / polePairs:.2f} rpm (极对数={polePairs})")

# 调用函数
IsTrackInEquU, Tem_max = calcIsTrackInEquU(
	We=We_test,
	mtpaTrack=mtpaTrack,
	fw1Track=fw1Track,
	mtpvTrack=mtpvTrack,
	UPmax=UPmax,
	IPmax=IPmax,
	id_demag=id_demag,
	equUPoints=200,
	splitRatio=10
)

print(f"\n✓ 计算完成！")
print(f"  等电压椭圆点数: {len(IsTrackInEquU)}")
if len(IsTrackInEquU) > 0:
	print(f"  等电压椭圆范围:")
	print(f"    iq: [{IsTrackInEquU[:, 0].min():.2f}, {IsTrackInEquU[:, 0].max():.2f}] A")
	print(f"    id: [{IsTrackInEquU[:, 1].min():.2f}, {IsTrackInEquU[:, 1].max():.2f}] A")
	print(f"  最大转矩: {Tem_max:.2f} N·m")
	
	# 找到最大转矩对应的id、iq
	max_tem_idx = 0
	for i in range(len(IsTrackInEquU)):
		Tem_i = calcTem(IsTrackInEquU[i, 1], IsTrackInEquU[i, 0])
		if Tem_i > calcTem(IsTrackInEquU[max_tem_idx, 1], IsTrackInEquU[max_tem_idx, 0]):
			max_tem_idx = i
	
	id_maxTem = IsTrackInEquU[max_tem_idx, 1]
	iq_maxTem = IsTrackInEquU[max_tem_idx, 0]
	
	print(f"  最大转矩点:")
	print(f"    iq = {iq_maxTem:.2f} A")
	print(f"    id = {id_maxTem:.2f} A")
	
	# 绘制等电压椭圆
	plt.figure(figsize=(10, 8))
	plt.plot(IsTrackInEquU[:, 1], IsTrackInEquU[:, 0], 'b-o', markersize=4, label='等电压椭圆')
	plt.plot(id_maxTem, iq_maxTem, 'r*', markersize=15, label=f'最大转矩点 ({Tem_max:.0f} N·m)')
	
	# 绘制完整IsTrack轨迹
	plt.plot(IsRange[:, 1], IsRange[:, 0], 'k--', alpha=0.3, label='完整电流轨迹')
	
	plt.xlabel('id (A)')
	plt.ylabel('iq (A)')
	plt.title(f'等电压椭圆 (We={We_test:.0f} rad/s, Us={Us_test:.0f} V)')
	plt.grid(True)
	plt.axhline(y=0, color='k', linestyle='--', alpha=0.3)
	plt.axvline(x=0, color='k', linestyle='--', alpha=0.3)
	plt.legend()
	plt.tight_layout()
	plt.show()
else:
	print("  等电压椭圆为空（可能转速过低）")

#### 2.3.8 测试函数：绘制电流控制范围及等电压/等转矩轨迹
- 绘制电流控制范围轨迹IsRange
- 绘制电流控制范围内的等电压椭圆轨迹IsTrackInEquU
	- 读取fw1Track的第一个点，获得最小转速minWe
	- 读取mtpvTrack的最后一个点，获得最大转速maxWe
	- 从minWe到maxWe，生成equUTracksNum长度转速序列（不包含We=minWe和maxWe）
	- 对每个We计算等电压椭圆
- 绘制电流控制范围内的等转矩轨迹
	- 读取mtpaTrack的最后一个点，获得最大转矩maxTem
	- 从0到maxTem，生成equTemTracksNum长度转矩序列（不包含Tem=0）
	- 对每个Tem计算等转矩轨迹
	- 绘制每个等转矩轨迹

In [ ]:
# type: ignore
def plotCurrentTracks(equUTracksNum: int,
					  equUPoints: int,
					  equTemTracksNum: int,
					  equTemPoints: int,
					  figsize: tuple = (14, 12)):
	"""
	绘制完整电流轨迹示意图
	
	参数:
		equUTracksNum: 等电压椭圆轨迹数量
		equUPoints: 等电压椭圆扫描点数
		equTemTracksNum: 等转矩线数量
		equTemPoints: 等转矩线扫描点数
		figsize: 图形大小
	"""

	# 调用前序函数
	UPmax, IPmax = calcVILimit(Udc, Imax, connectType='Y', modulationType='SVPWM')
	isDemag, id_demag = checkAndCalcDemagid(IPmax, deMagidPoints)
	mtpaTrack = calcIsTrackInMTPA(UPmax=UPmax, IPmax=IPmax, mtpaPoints=mtpaPoints)
	print(f"MTPA电流轨迹计算完成，点数: {len(mtpaTrack)}")
	if isDemag:
		mtpvTrack = calcMTPVIsTrack(We_base=We_base, UPmax=UPmax, IPmax=IPmax, id_demag=id_demag, 
									mtpvWeScanRatio=mtpvWeScanRatio, mtpvWePoints=mtpvWePoints )
		print(f"MTPV电流轨迹计算完成，点数: {len(mtpvTrack)}")
	else:
		mtpvTrack = None
		print(f"不发生完全去磁，跳过MTPV电流轨迹计算")
	fw1Track = calcFW1Track(mtpaTrack=mtpaTrack, mtpvTrack=mtpvTrack, UPmax=UPmax, IPmax=IPmax, fw1iqPoints=fw1iqPoints)
	print(f"弱磁I区电流轨迹计算完成，点数: {len(fw1Track)}")
	IsRange = calcFullTrack(mtpaTrack=mtpaTrack, mtpvTrack=mtpvTrack, fw1Track=fw1Track)
	print(f"完整电流轨迹计算完成，点数: {len(IsRange)}")

	# 获得转速范围
	minWe = fw1Track[0, 4]
	if isDemag:
		maxWe = mtpvTrack[-1, 4]
	else:
		maxWe = fw1Track[-1, 4]

	# 生成转速序列（按指数规律，在小转速取点密集，在大转速稀疏）
	# 使用以10为底的对数间距：在对数空间内均匀分布，转换回线性空间
	log_minWe = np.log(minWe)
	log_maxWe = np.log(maxWe)
	log_We_list = np.linspace(log_minWe, log_maxWe, equUTracksNum + 2)[1:-1]
	We_list = np.exp(log_We_list)
	
	# 获得最大转矩
	maxTem = mtpaTrack[-1, 3]
	
	# 生成转矩序列（不包含Tem=0）
	Tem_list = np.linspace(0, maxTem, equTemTracksNum + 1)[1:]  # 从0到maxTem，不包含0
	
	print(f"开始绘制电流轨迹示意图...")
	print(f"转速范围: {minWe:.2f} ~ {maxWe:.2f} rad/s")
	print(f"等电压椭圆数量: {equUTracksNum}")
	print(f"转速序列: {We_list}")
	print(f"最大转矩: {maxTem:.2f} N·m")
	print(f"等转矩线数量: {equTemTracksNum}")
	print(f"转矩序列: {Tem_list}")
	
	# 创建图形
	fig = plt.figure(figsize=figsize)
	
	# 首先绘制完整电流轨迹IsRange
	plt.plot(IsRange[:, 1], IsRange[:, 0], 'b-', linewidth=2.5, label='可利用的电流矢量范围', zorder=10)
	
	# 在IsRange上标注关键点
	# MTPA区的最后一点
	plt.plot(mtpaTrack[-1, 1], mtpaTrack[-1, 0], 'ro', markersize=8, label='MTPA终点', zorder=11)
	# 弱磁I区的起点和终点
	plt.plot(fw1Track[0, 1], fw1Track[0, 0], 'go', markersize=8, label='弱磁I区起点', zorder=11)
	plt.plot(fw1Track[-1, 1], fw1Track[-1, 0], 'mo', markersize=8, label='弱磁I区终点', zorder=11)
	
	# 绘制电流圆限制（Is = IPmax）
	theta = np.linspace(0, np.pi/2, 100)
	id_circle = -IPmax * np.cos(theta)
	iq_circle = IPmax * np.sin(theta)
	plt.plot(id_circle, iq_circle, 'k--', linewidth=1.5, alpha=0.6, label=f'电流限制圈 (Is={IPmax:.0f}A)')
	
	# ========== 绘制等电压椭圆轨迹 ==========
	print(f"\n[等电压椭圆轨迹] 开始计算...")
	colors_u = plt.cm.get_cmap('cool')(np.linspace(0, 1, equUTracksNum))
	
	for i, We_target in enumerate(We_list):
		print(f"正在计算第 {i+1}/{equUTracksNum} 条等电压椭圆，We = {We_target:.2f} rad/s")
		
		try:
			# 计算等电压椭圆轨迹
			equUTrack, Tem_max = calcIsTrackInEquU(
				We=We_target,
				mtpaTrack=mtpaTrack,
				fw1Track=fw1Track,
				mtpvTrack=mtpvTrack,
				UPmax=UPmax,
				IPmax=IPmax,
				id_demag=id_demag,
				equUPoints=equUPoints
			)
			
			# 绘制等电压椭圆
			if len(equUTrack) > 0:
				plt.plot(equUTrack[:, 1], equUTrack[:, 0], 
						color=colors_u[i], linewidth=1, alpha=0.4,
						label=f'等电压椭圆 {We_target:.0f} rad/s' if i % 2 == 0 else '')
				print(f"  ✓ 成功绘制，点数: {len(equUTrack)}, 最大转矩: {Tem_max:.2f} N·m")
			else:
				print(f"  ✗ 等电压椭圆轨迹为空")
		except Exception as e:
			print(f"  ✗ 计算失败: {e}")
	
	# ========== 绘制等转矩线 ==========

	print(f"\n[等转矩线] 开始计算...")
	colors_t = plt.cm.get_cmap('hot')(np.linspace(0, 1, equTemTracksNum))
	
	for i, Tem_target in enumerate(Tem_list):
		print(f"正在计算第 {i+1}/{equTemTracksNum} 条等转矩线，Tem = {Tem_target:.2f} N·m")
		
		try:
			# 计算等转矩轨迹
			equTemTrack = calcIsTrackInEquTem(
				Tem_target=Tem_target,
				UPmax=UPmax,
				IPmax=IPmax,
				mtpaTrack=mtpaTrack,
				fw1Track=fw1Track,
				mtpvTrack=mtpvTrack,
				equTemPoints=equTemPoints
			)
			
			# 绘制等转矩线
			if len(equTemTrack) > 0:
				plt.plot(equTemTrack[:, 1], equTemTrack[:, 0], 
						color=colors_t[i], linewidth=1, alpha=0.4,
						label=f'等转矩线 {Tem_target:.0f} N·m' if i % 2 == 0 else '')
				print(f"  ✓ 成功绘制，点数: {len(equTemTrack)}")
			else:
				print(f"  ✗ 等转矩轨迹为空")
		except Exception as e:
			print(f"  ✗ 计算失败: {e}")

	
	# 设置图形属性
	plt.xlabel('id (A)', fontsize=12)
	plt.ylabel('iq (A)', fontsize=12)
	plt.title('永磁同步电机电流轨迹示意图', fontsize=14)
	plt.grid(True, alpha=0.3)
	plt.axhline(y=0, color='k', linestyle='--', alpha=0.3, linewidth=0.5)
	plt.axvline(x=0, color='k', linestyle='--', alpha=0.3, linewidth=0.5)
	plt.legend(loc='upper right', fontsize=8, ncol=2)
	plt.axis('equal')
	plt.tight_layout()
	
	print(f"\n✓ 电流轨迹示意图绘制完成！")
	
	return fig

print("plotCurrentTracks函数定义完成！")

In [ ]:
# 测试：绘制完整电流轨迹示意图
fig = plotCurrentTracks(
	equUTracksNum=20,
	equUPoints=100,
	equTemTracksNum=equTemTracksNum,
	equTemPoints=equTemPoints,
	figsize=(14, 12)
)

plt.show()

## 3 指定工况点和电流控制策略，输出电流轨迹线
- 输入：目标转矩`Tem_target`、目标转速`We_target`、电流控制策略`controlStrategy（'id=0', 'MTPA'、'FW'）`、负载点类型`loadType（'constTem', 'constWe'）`
- 输出：
	- 可利用的电流范围轨迹`IsRange`:（格式：`[[iq0, id0], [iq1, id1], ...]`）
	- 最大容量下的电流轨迹`IsTrackAtMaxCapacity`：（格式：`[[iq0, id0，Is0, Tem0, We0], [iq1, id1, Is1, Tem1, We1], ...]`）
	- 恒转矩下的电流轨迹`equTemTrack`: （对应等转矩线，等转矩轨迹，格式：`[[iq0, id0, Tem0, We0], [iq1, id1, Tem1, We1], ...]`）
	- 恒转速下的电流轨迹`equWeTrack`（对应等电压椭圆`IsTrackInEquU`，等电压椭圆电流轨迹 (N×2)，格式：`[[iq0, id0，Tem0, We0], [iq1, id1, Tem1, We1], ...]`）
	- 给定工况对应的控制点（`[iq, id, Tem, We]`）

### 3.1 id=0 和 MTPA 控制策略的轨迹计算

#### 3.1.1 核心概念

给定目标转矩 `temTarget` 和转速 `weTarget`，本模块输出四类控制轨迹和工作点：

| 轨迹类型 | 用途 | 输出维度 |
|---------|------|--------|
| **IsRange** | 电流矢量的可用范围（边界约束） | N×2: [iq, id] |
| **IsTrackAtMaxCapacity** | 电机各转速下的最大输出容量轨迹 | N×5: [iq, id, Is, Tem, We] |
| **equTemTrack** | 恒定转矩下的可控范围（We从0到最大） | N×4: [iq, id, Tem, We] |
| **equWeTrack** | 恒定转速下的可控范围（Tem从0到最大） | N×4: [iq, id, Tem, We] |
| **给定工况点** | 负载对应的实际工作点 | 1×4: [iq, id, Tem, We] |

**可选控制策略选择**：
- **id=0策略** 
- **MTPA策略** 

---

#### 3.1.2 轨迹计算流程

- **输出 IsRange（电流可用范围）**

  - **定义**：可用的电流矢量的集合，由电压和电流限幅决定。

  - **计算方法**：
	```
	Step 1: 选择非弱磁轨迹
		- id=0策略：nonFwIsTrack = calcid0Track(UPmax, IPmax, id0Points)
		- MTPA策略：nonFwIsTrack = calcMtpaTrack(UPmax, IPmax, mtpaPoints)

	Step 2: 拼接完整轨迹（包括弱磁段，若有）
		IsRange = calcFullTrack(
			mtpaTrack=nonFwIsTrack,
			mtpvTrack=None,         # id=0/MTPA 不涉及弱磁
			fw1Track=None
		)

	Step 3: 输出格式
		IsRange: [[iq₀, id₀], [iq₁, id₁], ..., [iqₙ, idₙ]]
	```

- **输出 IsTrackAtMaxCapacity（最大容量轨迹）**

  - **定义**：电机在转速从0到最高可达值过程中，每个转速对应的最大转矩轨迹。

  - **关键要点**：
	 - 起点：We=0 时的最大转矩点
	 - 基速：We 从 0 递增到 weBase，电磁转矩保持不变
	 - 终点：非弱磁轨迹末端点

  - **计算方法**：
	```
	Step 1: 反向排列非弱磁轨迹
		nonFwIsTrack_reversed = np.flipud(nonFwIsTrack)

	Step 2: 生成 We=0 的起始点
		pointWe0 = nonFwIsTrack_reversed[0].copy()
		pointWe0[We列] = 0           # 将转速设为 0
		IsTrackAtMaxCapacity = [pointWe0]

	Step 3: 在 We=0 到 weBase 之间插入等步长控制点
		weBase = nonFwIsTrack_reversed[0, We列]  # 第一个反向点的转速
		weInterpList = np.linspace(0, weBase, constTemPointsAtNonFW)
		
		对每个 We_i 生成新的控制点：
		- 复制首点（id、iq、Tem保持不变）
		- 替换 We = We_i
		- 添加到轨迹中

	Step 4: 拼接非弱磁轨迹的剩余部分
		IsTrackAtMaxCapacity += nonFwIsTrack_reversed

	Step 5: 输出格式
		IsTrackAtMaxCapacity: 
		[[iq₀, id₀, Is₀, Tem₀, 0],
		[iq₁, id₁, Is₁, Tem₁, We_interp₁],
		[iq₂, id₂, Is₂, Tem₂, We_interp₂],
		...,
		[iq_base, id_base, Is_base, Tem_base, We_base],
		...,
		[iqₙ, idₙ, Isₙ, Temₙ, We_max]]
	```

- **输出 equTemTrack（恒转矩轨迹）**

  - **定义**：转矩保持为 `temTarget` 的条件下，转速从 0 到最大可达值的控制轨迹。

  - **前提条件**：
	 - temTarget 必须在非弱磁可达范围内
	 - 超出范围时自动限幅至最大转矩

  - **计算方法**：
	```
	Step 1: 限幅目标转矩
		temMax = nonFwIsTrack[-1, Tem列]  # 非弱磁轨迹的最大转矩
		temTargetClamped = min(temTarget, temMax)

	Step 2: 在非弱磁轨迹中对目标转矩插值
		pointAtEquTem = interp_sorted(
			temTargetClamped,
			nonFwIsTrack[:, Tem列],       # 转矩数据列
			nonFwIsTrack                  # 返回完整行
		)

	Step 3: 构建恒转矩轨迹（转速等步长从0递增）
		weMaxAtEquTem = pointAtEquTem[We列]  # 在目标转矩处的最大转速
		WeList = np.linspace(0, weMaxAtEquTem, constTemPointsAtNonFW)
		
		equTemTrack = []
		对每个 We_i：
		- 复制 pointAtEquTem 的全部数据
		- 更新 We = We_i
		- 仅提取 [iq, id, temTargetClamped,We_i] 添加到 equTemTrack

	Step 4: 输出格式（提取的三列数据）
		equTemTrack (N×4): 
		[[iq, id, temTargetClamped, 0],
		[iq, id, temTargetClamped, We_interp₁],
		[iq, id, temTargetClamped, We_interp₂],
		...,
		[iq, id, temTargetClamped, We_max]]
		
		说明：iq和id保持恒定（对应 temTarget），We从0等步长递增至最大
	```

- **输出 equWeTrack（恒转速轨迹）**

  - **定义**：转速保持为 `weTarget` 的条件下，从零转矩到最大可达转矩的控制轨迹。

  - **前提条件**：
	 - weTarget 必须在非弱磁可达范围内
	 - 超出范围时自动限幅至最大转速

  - **计算方法**：
	```
	Step 1: 限幅目标转速
		WeMax = nonFwIsTrack[0, We列]  # 非弱磁轨迹的最大转速
		weTargetClamped = min(weTarget, weMax)

	Step 2: 在非弱磁轨迹中对目标转速插值
		pointAtEquWe = interp_sorted(
			weTargetClamped,
			nonFwIsTrack[:, We列],        # 转速数据列
			nonFwIsTrack
		)

	Step 3: 根据插值点的电流约束重新计算完整非弱磁轨迹
		Is_at_we = pointAtEquWe[Is列]  # 在目标转速处的电流幅值
		
		if controlType == 'mtpa':
			equWeTrack = calcIsTrackInMTPA(
				UPmax=UPmax, 
				IPmax=Is_at_we, 
				mtpaPoints=constWePointsAtNonFW
			)
		else:
			equWeTrack = calcid0Track(
				UPmax=UPmax, 
				IPmax=Is_at_we, 
				id0Points=constWePointsAtNonFW
			)
		
		说明：以目标转速处的电流幅值作为新的上限约束，
		重新生成非弱磁轨迹（从零至最大转矩）

	Step 4: 提取并输出格式为 [iq, id, Tem, weTargetClamped]
		equWeTrack: 
		[[iq₀, id₀, Tem₀, weTargetClamped],
		[iq₁, id₁, Tem₁, weTargetClamped],
		...,
		[iqₙ, idₙ, Tem_max, weTargetClamped]]
	```

#### 3.1.3 给定工况点的计算

- **输入**：`loadType`（负载类型）、`constTem`、`constWe`

- **输出**：`givenPoint = [iq, id, Tem, We]`

- **情况 A：恒转矩负载 (`loadType='constTem'`)**

	```
	前提：已获得 equTemTrack（恒转矩轨迹）

	步骤：
	1. 检查目标转速是否可达，转速超出范围 → 取轨迹的最大We点
		if weTarget > weMaxAtEquTem:
			givenPoint = equTemTrack[-1]
		
	3. 转速在范围内 → 在轨迹中插值
		else:
			givenPoint = interp_sorted(
				weTarget,
				equTemTrack[:, We列],
				equTemTrack
			)

	输出：[iq_given, id_given, Tem_target, We_given]
	```

- **情况 B：恒转速负载 (`loadType='constWe'`)**

	```
	前提：已获得 equWeTrack（恒转速轨迹）

	步骤：
	1. 检查目标转矩是否可达，转矩超出范围 → 取轨迹的最大Tem点
		if temTarget > temMaxAtEquWe:
			givenPoint = equWeTrack[-1]
		
	3. 转矩在范围内 → 在轨迹中插值
		else:
			givenPoint = interp_sorted(
				temTarget,
				equWeTrack[:, Tem列],
				equWeTrack
			)

	输出：[iq_given, id_given, Tem_given, We_target]
	```

---

#### 3.1.4 总体计算流程图

```
输入参数：temTarget, weTarget, loadType

生成非弱磁轨迹 (nonFwIsTrack)
│
├─→ 输出 1: IsRange
│   非弱磁轨迹的 [iq, id]
│
├─→ 输出 2: IsTrackAtMaxCapacity
│   • We=0 的起点
│   • We 等步长插值段
│   • 非弱磁轨迹段
│
├─→ 输出 3: equTemTrack (恒转矩轨迹)
│   • 在非弱磁轨迹中对 temTarget 插值获得 pointAtEquTem
│   • We 从 0 等步长变化到pointAtEquTem对应的最大 weMaxAtEquTem
│   • 每个 We 对应一个控制点（保持Id/Iq/Tem不变）
│
├─→ 输出 4: equWeTrack (恒转速轨迹)
│   • 在非弱磁轨迹中对 weTarget 插值获得 pointAtEquWe
│   • 以 pointAtEquWe 的电流幅值(Is)为上限，重新计算非弱磁轨迹
│   • 输出为 [iq, id, Tem] 格式（Tem 从 0 递增至最大）
│
└─→ 输出 5: given_point (工作点)
	 if loadType='constTem':
		  从 equTemTrack 对 weTarget 插值
	 else:
		  从 equWeTrack 对 temTarget 插值
```

In [ ]:
# type: ignore
def generateNonFwResult(controlType, temTarget, weTarget, loadType):
	"""
	根据非弱磁轨迹，计算id=0/MTPA控制策略下的四类轨迹和工作点

	参数：
	------
	controlType : str
		控制策略，'id0'（id=0）或 'mtpa'（MTPA）
	temTarget : float
		目标转矩
	weTarget : float
		目标转速（电角速度）
	loadType : str
		负载类型，'constTem'（恒转矩）或 'constWe'（恒转速）

	返回值：
	------
	dict 包含：
		'IsRange': np.ndarray (N×2) - [iq, id] 电流可用范围
		'IsTrackAtMaxCapacity': np.ndarray - [iq, id, Is, Tem, We] 最大容量轨迹
		'equTemTrack': np.ndarray (M×4) - [iq, id, Tem, We] 恒转矩轨迹
		'equWeTrack': np.ndarray (M×4) - [iq, id, Tem, We] 恒转速轨迹
		'givenPoint': np.ndarray (1×4) - [iq, id, Tem, We] 工作点
	"""

	# 列索引定义
	iqIdx, idIdx, IsIdx, temIdx, weIdx = 0, 1, 2, 3, 4

	# ========================================
	# 1. IsRange - 电流可用范围
	# ========================================
	if controlType == 'id0':
		nonFwIsTrack = calcid0Track(UPmax=UPmax, IPmax=IPmax, id0Points=id0Points)
	elif controlType == 'mtpa':
		nonFwIsTrack = calcIsTrackInMTPA(UPmax=UPmax, IPmax=IPmax, mtpaPoints=mtpaPoints)
	else:
		raise ValueError(f"不支持的控制策略: {controlType}")
	IsRange = calcFullTrack(mtpaTrack=mtpaTrack, mtpvTrack=None, fw1Track=None)

	# ========================================
	# 2. IsTrackAtMaxCapacity - 最大容量轨迹
	# ========================================
	# Step 1: 反向排列非弱磁轨迹
	nonFwIsTrack_reversed = np.flipud(nonFwIsTrack)

	# Step 2: 生成 We=0 的起始点
	pointWe0 = nonFwIsTrack_reversed[0].copy()
	pointWe0[weIdx] = 0
	IsTrackAtMaxCapacity = [pointWe0]

	# Step 3: 在 We=0 到 weBase 之间插入等步长控制点
	weBase = nonFwIsTrack_reversed[0, weIdx]
	weInterpList = np.linspace(0, weBase, constTemPointsAtNonFW)

	for i, We_i in enumerate(weInterpList):
		if i == 0 or i == len(weInterpList) - 1:
			continue  # 跳过起点和终点（已在其他地方处理）
		pointInterp = nonFwIsTrack_reversed[0].copy()
		pointInterp[weIdx] = We_i
		IsTrackAtMaxCapacity.append(pointInterp)

	# Step 4: 拼接剩余轨迹
	IsTrackAtMaxCapacity.extend(nonFwIsTrack_reversed)
	IsTrackAtMaxCapacity = np.array(IsTrackAtMaxCapacity)

	# ========================================
	# 3. equTemTrack - 恒转矩轨迹
	# ========================================
	# Step 1: 限幅目标转矩
	temMax = nonFwIsTrack[-1, temIdx]
	temTargetClamped = min(temTarget, temMax)

	# Step 2: 在非弱磁轨迹中对目标转矩插值
	pointAtEquTem = interp_sorted(
		temTargetClamped,
		nonFwIsTrack[:, temIdx],
		nonFwIsTrack
	)

	# Step 3: 构建恒转矩轨迹
	weMaxAtEquTem = pointAtEquTem[weIdx]
	weList = np.linspace(0, weMaxAtEquTem, constTemPointsAtNonFW)

	equTemTrack = []
	for weI in weList:
		point = pointAtEquTem.copy()
		point[weIdx] = weI
		equTemTrack.append(point[[iqIdx, idIdx, temIdx, weIdx]])
	equTemTrack = np.array(equTemTrack)

	# ========================================
	# 4. equWeTrack - 恒转速轨迹
	# ========================================
	# Step 1: 限幅目标转速
	weMax = nonFwIsTrack[0, weIdx]
	weTargetClamped = min(weTarget, weMax)

	# Step 2: 在非弱磁轨迹中对目标转速插值
	pointAtEquWe = interp_sorted(
		weTargetClamped,
		nonFwIsTrack[:, weIdx],
		nonFwIsTrack
	)
	temMaxAtEquWe = pointAtEquWe[temIdx]

	# Step 3: 在pointAtEquWe[isIdx]电流限值下，重新计算非弱磁电流轨迹
	if controlType == 'mtpa':
		equWeTrack = calcIsTrackInMTPA(UPmax=UPmax, IPmax=pointAtEquWe[IsIdx], mtpaPoints=constWePointsAtNonFW)
	else:
		equWeTrack = calcid0Track(UPmax=UPmax, IPmax=pointAtEquWe[IsIdx], id0Points=constWePointsAtNonFW)

	# step 4: 将重新计算出的轨迹中的 We 替换为目标转速
	equWeTrack[:, weIdx] = weTargetClamped
	equWeTrack = equWeTrack[:, [iqIdx, idIdx, temIdx, weIdx]]
	equWeTrack = np.array(equWeTrack)

	# ========================================
	# 5. 给定工况点的计算
	# ========================================
	if loadType == 'constTem':
		# 情况 A：恒转矩负载
		if weTarget > weMaxAtEquTem:
			givenPoint = equTemTrack[-1].copy()
			print(f'恒转矩负载，目标转速超出最大转速\n')
		else:
			givenPoint = interp_sorted(
				weTarget,
				equTemTrack[:, 3],
				equTemTrack
			)

	elif loadType == 'constWe':
		# 情况 B：恒转速负载
		if temTarget > temMaxAtEquWe:
			givenPoint = equWeTrack[-1].copy()
			print(f'恒转速负载，目标转矩超出最大转矩\n')
		else:
			givenPoint = interp_sorted(
				temTarget,
				equWeTrack[:, 2],
				equWeTrack
			)
	else:
		raise ValueError(f"不支持的负载类型: {loadType}")

	# ========================================
	# 返回结果
	# ========================================
	return {
		'IsRange': IsRange,
		'IsTrackAtMaxCapacity': IsTrackAtMaxCapacity,
		'equTemTrack': equTemTrack,
		'equWeTrack': equWeTrack,
		'givenPoint': givenPoint
	}

### 3.2 弱磁控制策略的轨迹计算

#### 3.2.1 轨迹计算流程

- **输出 IsRange（电流可用范围）**

  - **定义**：可用的电流矢量的集合，由电压和电流限幅决定。

  - **计算方法**：
	```
	Step 1: 计算非弱磁轨迹
		- id=0策略：nonFwIsTrack = calcid0Track(UPmax, IPmax, id0Points)
		- MTPA策略：nonFwIsTrack = calcMtpaTrack(UPmax, IPmax, mtpaPoints)
	
	Step 2: 计算MTPV轨迹
		nonFwIsTrack_end = nonFwIsTrack[-1]
		mtpvTrack = calcMtpvTrack(
			We_base=nonFwIsTrack_end[We列],
			mtpvWeScanRatio=mtpvWeScanRatio,
			UPmax=UPmax,
			IPmax=IPmax,
			id_demag=id_demag,
			mtpvPoints=mtpvPoints
		)
	
	Step 3: 计算弱磁I区轨迹
		fw1Track = calcFw1Track(
			mtpaTrack=nonFwIsTrack,
			mtpvTrack=mtpvTrack,
			UPmax=UPmax,
			IPmax=IPmax,
			fw1Points=fw1Points
		)

	Step 4: 拼接完整轨迹
		IsRange = calcFullTrack(
			mtpaTrack=nonFwIsTrack,
			mtpvTrack=mtpvTrack,
			fw1Track=fw1Track
		)

	Step 5: 输出格式
		IsRange: [[iq₀, id₀], [iq₁, id₁], ..., [iqₙ, idₙ]]
	```

- **输出 IsTrackAtMaxCapacity（最大容量轨迹）**

  - **定义**：电机在转速从0到最高可达值过程中，每个转速对应的最大转矩轨迹。

  - **关键要点**：
	 - 起点：We=0 时的最大转矩点
	 - 基速：We 从 0 递增到 weBase，电磁转矩保持不变
	 - 终点：非弱磁轨迹末端点

  - **计算方法**：
	```
	Step 1: 反向排列非弱磁轨迹
		nonFwIsTrack_reversed = np.flipud(nonFwIsTrack)

	Step 2: 生成 We=0 的起始点
		pointWe0 = nonFwIsTrack_reversed[0].copy()
		pointWe0[We列] = 0           # 将转速设为 0
		IsTrackAtMaxCapacity = [pointWe0]

	Step 3: 在 We=0 到 weBase 之间插入等步长控制点
		weBase = nonFwIsTrack_reversed[0, We列]  # 第一个反向点的转速
		weInterpList = np.linspace(0, weBase, constTemPointsAtNonFW)
		
		对每个 We_i 生成新的控制点：
		- 复制首点（id、iq、Tem保持不变）
		- 替换 We = We_i
		- 添加到轨迹中

	Step 4: 拼接FW1轨迹+MTPV轨迹(如果不为None)
		IsTrackAtMaxCapacity += fw1Track
		if mtpvTrack is not None:
			IsTrackAtMaxCapacity += mtpvTrack
	```

- **输出 equTemTrack（恒转矩轨迹）**

  - **定义**：转矩保持为 `temTarget` 的条件下，转速从 0 到最大可达值的控制轨迹。

  - **前提条件**：
	 - temTarget 必须在非弱磁可达范围内
	 - 超出范围时自动限幅至最大转矩

  - **计算方法**：
	```
	Step 1: 限幅目标转矩
		temMax = nonFwIsTrack[-1, Tem列]  # 非弱磁轨迹的最大转矩，对应IsTrackAtMaxCapacity上的最大转矩
		temTargetClamped = min(temTarget, temMax)

	Step 2: 在IsTrackAtMaxCapacity中对目标转矩插值，获得最大转速点maxWePointAtEquTem
		maxWePointAtEquTem = interp_sorted(
			temTargetClamped,
			IsTrackAtMaxCapacity[:, Tem列],       # 转矩数据列
			IsTrackAtMaxCapacity                  # 返回完整行
		)

	Step 3: 在nonFwIsTrack中对目标转矩插值，获得弱磁分界点fwBoundaryPointAtEquTem
		fwBoundaryPointAtEquTem = interp_sorted(
			temTargetClamped,
			nonFwIsTrack[:, Tem列],       # 转矩数据列
			nonFwIsTrack                  # 返回完整行
		)

	Step 4: 构建恒转矩轨迹-非弱磁段（转速等步长从0递增）
		fwBoundaryWe = fwBoundaryPointAtEquTem[We列]  # 在目标转矩处的最大转速
		WeList = np.linspace(0, fwBoundaryWe, constTemPointsAtNonFW)
		
		equTemTrack = []
		对每个 We_i：
		- 复制 fwBoundaryPointAtEquTem 的全部数据
		- 更新 We = We_i

	Step 5: 计算弱磁段的等转矩轨迹
		equTemTrack_fw = calcIsTrackInEquTem(
			Tem_target=temTargetClamped,
			UPmax=UPmax,
			IPmax=IPmax,
			mtpaTrack=nonFwIsTrack,
			fw1Track=fw1Track,
			mtpvTrack=mtpvTrack,
			equTemPoints=constTemPointsAtFW
		)

	step 6: 拼接非弱磁段和弱磁段
		equTemTrack += equTemTrack_fw

	Step 7: 提取出[iq, id, Tem, We]格式
		equTemTrack = [
			[point[iq列], point[id列], point[Tem列], point[We列]]
			for point in equTemTrack
		]
	```

- **输出 equWeTrack（恒转速轨迹）**

  - **定义**：转速保持为 `weTarget` 的条件下，从零转矩到最大可达转矩的控制轨迹。

  - **前提条件**：
	 - weTarget 必须在非弱磁可达范围内
	 - 超出范围时自动限幅至最大转速

  - **计算方法**：
	```
	Step 1: 限幅目标转速
		WeMax = IsTrackAtMaxCapacity[-1, We列]  # 对应IsTrackAtMaxCapacity上的最大转速
		weTargetClamped = min(weTarget, weMax)

	Step 2: 在非弱磁轨迹中对目标转速插值
		maxTemPointAtEquWe = interp_sorted(
			weTargetClamped,
			IsTrackAtMaxCapacity[:, We列],        # 转速数据列
			IsTrackAtMaxCapacity
		)

	Step 3: 判断weTargetClamped是否与非弱磁轨迹有交点（即weTargetClamped<nonFwIsTrack[0, We列]），如果有交点，直接在nonFwIsTrack中插值获得fwBoundaryPointAtEquWe, 否则fwBoundaryPointAtEquWe为None
	
	Step4: 如果fwBoundaryPointAtEquWe不为None，根据插值点的电流约束重新计算完整非弱磁轨迹
		IsAtFwBoundaryPoint = fwBoundaryPointAtEquWe[Is列]  
		
		if controlType == 'mtpa':
			equWeTrack = calcIsTrackInMTPA(
				UPmax=UPmax, 
				IPmax=IsAtFwBoundaryPoint, 
				mtpaPoints=constWePointsAtNonFW
			)
		else:
			equWeTrack = calcid0Track(
				UPmax=UPmax, 
				IPmax=IsAtFwBoundaryPoint, 
				id0Points=constWePointsAtNonFW
			)
		
		将equWeTrack的We值全部设为weTargetClamped
		for point in equWeTrack:
			point[We列] = weTargetClamped

	否则，equWeTrack=[]

	Step 5: 计算弱磁段的等转速轨迹
		equWeTrack_fw = calcIsTrackInEquU(
			We=weTargetClamped,
			mtpaTrack=nonFwIsTrack,
			fw1Track=fw1Track,
			mtpvTrack=mtpvTrack,    
			UPmax=UPmax,
			IPmax=IPmax,
			id_demag=id_demag,
			equUPPoints=constWePointsAtFW,
			splitRatio=5
		)[0]
	
	step 6: 拼接非弱磁段和弱磁段
		equWeTrack += equWeTrack_fw
	
	Step 7: 提取出[iq, id, Tem, We]格式
	```

#### 3.1.3 给定工况点的计算

- **输入**：`loadType`（负载类型）、`constTem`、`constWe`

- **输出**：`givenPoint = [iq, id, Tem, We]`

- **情况 A：恒转矩负载 (`loadType='constTem'`)**

	```
	前提：已获得 equTemTrack（恒转矩轨迹）

	步骤：
	1. 检查目标转速是否可达，转速超出范围 → 取轨迹的最大We点
		if weTarget > equTemTrack[-1][We列]:
			givenPoint = equTemTrack[-1]
		
	3. 转速在范围内 → 在轨迹中插值
		else:
			givenPoint = interp_sorted(
				weTarget,
				equTemTrack[:, We列],
				equTemTrack
			)

	输出：[iq_given, id_given, Tem_target, We_given]
	```

- **情况 B：恒转速负载 (`loadType='constWe'`)**

	```
	前提：已获得 equWeTrack（恒转速轨迹）

	步骤：
	1. 检查目标转矩是否可达，转矩超出范围 → 取轨迹的最大Tem点
		if temTarget > equWeTrack[-1][Tem列]:
			givenPoint = equWeTrack[-1]
		
	3. 转矩在范围内 → 在轨迹中插值
		else:
			givenPoint = interp_sorted(
				temTarget,
				equWeTrack[:, Tem列],
				equWeTrack
			)

	输出：[iq_given, id_given, Tem_given, We_target]
	```

In [ ]:
# type: ignore
def generateFwResult(controlType, temTarget, weTarget, loadType):
	"""
	根据弱磁轨迹，计算id=0/MTPA控制策略下的四类轨迹和工作点

	参数：
	------
	controlType : str
		控制策略，'id0'（id=0）或 'mtpa'（MTPA）
	temTarget : float
		目标转矩
	weTarget : float
		目标转速（电角速度）
	loadType : str
		负载类型，'constTem'（恒转矩）或 'constWe'（恒转速）

	返回值：
	------
	dict 包含：
		'IsRange': np.ndarray (N×2) - [iq, id] 电流可用范围
		'IsTrackAtMaxCapacity': np.ndarray - [iq, id, Is, Tem, We] 最大容量轨迹
		'equTemTrack': np.ndarray (M×4) - [iq, id, Tem, We] 恒转矩轨迹
		'equWeTrack': np.ndarray (M×4) - [iq, id, Tem, We] 恒转速轨迹
		'givenPoint': np.ndarray (1×4) - [iq, id, Tem, We] 工作点
	"""

	# 列索引定义
	iqIdx, idIdx, IsIdx, temIdx, weIdx = 0, 1, 2, 3, 4

	# ========================================
	# 1. IsRange - 电流可用范围
	# ========================================
	# Step 1: 计算非弱磁轨迹
	if controlType == 'id0':
		nonFwIsTrack = calcid0Track(UPmax=UPmax, IPmax=IPmax, id0Points=id0Points)
	elif controlType == 'mtpa':
		nonFwIsTrack = calcIsTrackInMTPA(UPmax=UPmax, IPmax=IPmax, mtpaPoints=mtpaPoints)
	else:
		raise ValueError(f"不支持的控制策略: {controlType}")
	
	# Step 2: 计算MTPV轨迹
	nonFwIsTrack_end = nonFwIsTrack[-1]
	mtpvTrack = calcMTPVIsTrack(
		We_base=nonFwIsTrack_end[weIdx],
		mtpvWeScanRatio=mtpvWeScanRatio,
		UPmax=UPmax,
		IPmax=IPmax,
		id_demag=id_demag,
		mtpvWePoints=mtpvWePoints
	)
	
	# Step 3: 计算弱磁I区轨迹
	fw1Track = calcFW1Track(
		mtpaTrack=nonFwIsTrack,
		mtpvTrack=mtpvTrack,
		UPmax=UPmax,
		IPmax=IPmax,
		fw1iqPoints=fw1iqPoints
	)

	# Step 4: 拼接完整轨迹
	IsRange = calcFullTrack(
		mtpaTrack=nonFwIsTrack,
		mtpvTrack=mtpvTrack,
		fw1Track=fw1Track
	)

	# ========================================
	# 2. IsTrackAtMaxCapacity - 最大容量轨迹
	# ========================================
	# Step 1: 反向排列非弱磁轨迹
	nonFwIsTrack_reversed = np.flipud(nonFwIsTrack)

	# Step 2: 生成 We=0 的起始点
	pointWe0 = nonFwIsTrack_reversed[0].copy()
	pointWe0[weIdx] = 0
	IsTrackAtMaxCapacity = [pointWe0]

	# Step 3: 在 We=0 到 weBase 之间插入等步长控制点
	weBase = nonFwIsTrack_reversed[0, weIdx]
	weInterpList = np.linspace(0, weBase, constTemPointsAtNonFW)

	for i, We_i in enumerate(weInterpList):
		if i == 0 or i == len(weInterpList) - 1:
			continue  # 跳过起点和终点（已在其他地方处理）
		pointInterp = nonFwIsTrack_reversed[0].copy()
		pointInterp[weIdx] = We_i
		IsTrackAtMaxCapacity.append(pointInterp)

	# Step 4: 拼接弱磁轨迹fwIsTrack和最大容量轨迹IsTrackAtMaxCapacity
	fwIsTrack = fw1Track
	if mtpvTrack is not None:
		fwIsTrack = np.concatenate((fwIsTrack, mtpvTrack), axis=0)
	IsTrackAtMaxCapacity.extend(fwIsTrack.tolist())
	IsTrackAtMaxCapacity = np.array(IsTrackAtMaxCapacity)

	# ========================================
	# 3. equTemTrack - 恒转矩轨迹
	# ========================================
	# Step 1: 限幅目标转矩
	temMax = nonFwIsTrack[-1, temIdx]  # 非弱磁轨迹的最大转矩
	temTargetClamped = min(temTarget, temMax)

	# Step 2: 在fwIsTrack中对目标转矩插值，获得最大转速点
	maxWePointAtEquTem = interp_sorted(
		temTargetClamped,
		fwIsTrack[:, temIdx],
		fwIsTrack
	)

	# Step 3: 在nonFwIsTrack中对目标转矩插值，获得弱磁分界点
	fwBoundaryPointAtEquTem = interp_sorted(
		temTargetClamped,
		nonFwIsTrack[:, temIdx],
		nonFwIsTrack
	)

	# Step 4: 构建恒转矩轨迹-非弱磁段（转速等步长从0递增）
	fwBoundaryWe = fwBoundaryPointAtEquTem[weIdx]
	weList = np.linspace(0, fwBoundaryWe, constTemPointsAtNonFW)

	equTemTrack = []
	for weI in weList:
		point = fwBoundaryPointAtEquTem.copy()
		point[weIdx] = weI
		equTemTrack.append(point)

	# Step 5: 计算弱磁段的等转矩轨迹
	equTemTrack_fw = calcIsTrackInEquTem(
		Tem_target=temTargetClamped,
		UPmax=UPmax,
		IPmax=IPmax,
		mtpaTrack=nonFwIsTrack,
		fw1Track=fw1Track,
		mtpvTrack=mtpvTrack,
		equTemPoints=constTemPointsAtFW
	)

	# Step 6: 拼接非弱磁段和弱磁段
	equTemTrack.extend(equTemTrack_fw)
		
	# Step 7: 提取出[iq, id, Tem, We]格式
	equTemTrack = np.array([
		[point[iqIdx], point[idIdx], point[temIdx], point[weIdx]]
		for point in equTemTrack
	])

	# ========================================
	# 4. equWeTrack - 恒转速轨迹
	# ========================================
	# Step 1: 限幅目标转速
	weMax = IsTrackAtMaxCapacity[-1, weIdx]
	weTargetClamped = min(weTarget, weMax)

	# Step 2: 在fwIsTrack中对目标转速插值
	maxTemPointAtEquWe = interp_sorted(
		weTargetClamped,
		fwIsTrack[:, weIdx],
		fwIsTrack
	)

	# Step 3: 判断weTargetClamped是否与非弱磁轨迹有交点
	equWeTrack = []
	fwBoundaryPointAtEquWe = None
	
	if weTargetClamped < nonFwIsTrack[0, weIdx]:
		# 有交点，在nonFwIsTrack中插值
		fwBoundaryPointAtEquWe = interp_sorted(
			weTargetClamped,
			nonFwIsTrack[:, weIdx],
			nonFwIsTrack
		)

		# Step 4: 根据插值点的电流约束重新计算完整非弱磁轨迹
		IsAtFwBoundaryPoint = fwBoundaryPointAtEquWe[IsIdx]
		
		if controlType == 'mtpa':
			equWeTrack_nonFw = calcIsTrackInMTPA(
				UPmax=UPmax,
				IPmax=IsAtFwBoundaryPoint,
				mtpaPoints=constWePointsAtNonFW
			)
		else:
			equWeTrack_nonFw = calcid0Track(
				UPmax=UPmax,
				IPmax=IsAtFwBoundaryPoint,
				id0Points=constWePointsAtNonFW
			)
		
		# 将equWeTrack的We值全部设为weTargetClamped
		for point in equWeTrack_nonFw:
			point[weIdx] = weTargetClamped
		
		equWeTrack = list(equWeTrack_nonFw)

	# Step 5: 计算弱磁段的等转速轨迹
	equWeTrack_fw = calcIsTrackInEquU(
		We=weTargetClamped,
		mtpaTrack=nonFwIsTrack,
		fw1Track=fw1Track,
		mtpvTrack=mtpvTrack,
		UPmax=UPmax,
		IPmax=IPmax,
		id_demag=id_demag,
		equUPoints=constWePointsAtFW,
		splitRatio=5
	)[0]

	# Step 6: 拼接非弱磁段和弱磁段
	equWeTrack.extend(equWeTrack_fw)

	# Step 7: 提取出[iq, id, Tem, We]格式
	equWeTrack = np.array([
		[point[iqIdx], point[idIdx], point[temIdx], point[weIdx]]
		for point in equWeTrack
	])

	# ========================================
	# 5. 给定工况点的计算
	# ========================================
	if loadType == 'constTem':
		# 情况 A：恒转矩负载
		if weTarget > equTemTrack[-1, 3]:
			givenPoint = equTemTrack[-1].copy()
			print(f'恒转矩负载，目标转速超出最大转速\n')
		else:
			givenPoint = interp_sorted(
				weTarget,
				equTemTrack[:, 3],
				equTemTrack
			)

	elif loadType == 'constWe':
		# 情况 B：恒转速负载
		if temTarget > equWeTrack[-1, 2]:
			givenPoint = equWeTrack[-1].copy()
			print(f'恒转速负载，目标转矩超出最大转矩\n')
		else:
			givenPoint = interp_sorted(
				temTarget,
				equWeTrack[:, 2],
				equWeTrack
			)
	else:
		raise ValueError(f"不支持的负载类型: {loadType}")

	# ========================================
	# 返回结果
	# ========================================
	return {
		'IsRange': IsRange,
		'IsTrackAtMaxCapacity': IsTrackAtMaxCapacity,
		'equTemTrack': equTemTrack,
		'equWeTrack': equWeTrack,
		'givenPoint': givenPoint
	}

### 3.3 测试函数

**功能：** 测试和可视化 `generateNonFwResult` 函数的计算结果

**输入参数：**
- `result` (dict)：`generateNonFwResult` 函数的返回字典，包含以下键值：
	- `'IsRange'`: np.ndarray, shape (N, 2) → [iq, id] 电流控制范围
	- `'IsTrackAtMaxCapacity'`: np.ndarray, shape (N, 5) → [iq, id, Is, Tem, We] 最大容量轨迹
	- `'equTemTrack'`: np.ndarray, shape (N, 3) → [iq, id, We] 恒转矩轨迹
	- `'equWeTrack'`: np.ndarray, shape (N, 3) → [iq, id, Tem] 恒转速轨迹
	- `'givenPoint'`: np.ndarray, shape (4,) → [iq, id, Tem, We] 给定工况点

**输出：**
1. **控制台输出**：打印各轨迹的数据形状和前几个点的详细信息
2. **可视化图表**：在 We-Tem 平面绘制：
	 - 最大容量轨迹（蓝色实线）
	 - 恒转速轨迹（绿色方块）
	 - 恒转矩轨迹（红色三角）
	 - 给定工况点（黄边黑星）

**使用方法：**
```python
# 步骤1：调用主函数获取结果
result = generateNonFwResult(
		Tem_target=100,
		We_target=200,
		controlStrategy='MTPA',
		loadType='constTem'
)

# 步骤2：使用测试函数进行验证和可视化
test_id0_mtpa_result_visualization(result)
```

In [ ]:
def test_id0_mtpa_result_visualization(result):
	"""
	测试和可视化 id=0/MTPA 控制策略计算结果
	
	参数：
	------
	result : dict
		包含以下键值的字典：
		- 'IsRange': np.ndarray, shape (N, 2), [iq, id] 电流范围
		- 'IsTrackAtMaxCapacity': np.ndarray, shape (N, 5), [iq, id, Is, Tem, We] 最大容量轨迹
		- 'equTemTrack': np.ndarray, shape (N, 4), [iq, id, Tem, We] 恒转矩轨迹
		- 'equWeTrack': np.ndarray, shape (N, 4), [iq, id, Tem, We] 恒转速轨迹
		- 'givenPoint': np.ndarray, shape (4,), [iq, id, Tem, We] 给定工况点
	
	功能：
	------
	1. 打印并验证各轨迹数据的形状和前几个点
	2. 在 We-Tem 平面绘制所有轨迹和工况点
	"""
	
	# ==================== 验证输出 ====================
	print("\n" + "="*80)
	print("验证计算结果")
	print("="*80)
	print(f"\n输出键值: {list(result.keys())}")
	
	# 1. IsRange - 电流范围
	IsRange = result['IsRange']
	print(f"\n【1. 电流范围 (IsRange)】")
	print(f"  形状: {IsRange.shape}")
	print(f"  前3个点 [iq, id]:")
	for i in range(min(3, len(IsRange))):
		print(f"    [{i}] iq={IsRange[i,0]:7.3f} A, id={IsRange[i,1]:7.3f} A")
	
	# 2. IsTrackAtMaxCapacity - 最大容量轨迹
	IsTrackAtMaxCapacity = result['IsTrackAtMaxCapacity']
	print(f"\n【2. 最大容量轨迹 (IsTrackAtMaxCapacity)】")
	print(f"  形状: {IsTrackAtMaxCapacity.shape}")
	print(f"  前3个点 [iq, id, Is, Tem, We]:")
	for i in range(min(3, len(IsTrackAtMaxCapacity))):
		print(f"    [{i}] iq={IsTrackAtMaxCapacity[i,0]:7.3f} A, "
			  f"id={IsTrackAtMaxCapacity[i,1]:7.3f} A, "
			  f"Is={IsTrackAtMaxCapacity[i,2]:7.3f} A, "
			  f"Tem={IsTrackAtMaxCapacity[i,3]:6.1f} N·m, "
			  f"We={IsTrackAtMaxCapacity[i,4]:6.1f} rad/s")
	
	# 3. equTemTrack - 恒转矩轨迹
	equTemTrack = result['equTemTrack']
	print(f"\n【3. 恒转矩轨迹 (equTemTrack)】")
	print(f"  形状: {equTemTrack.shape}")
	print(f"  前3个点 [iq, id, We]:")
	for i in range(min(3, len(equTemTrack))):
		print(f"    [{i}] iq={equTemTrack[i,0]:7.3f} A, "
			  f"id={equTemTrack[i,1]:7.3f} A, "
			  f"Tem={equTemTrack[i,2]:6.1f} N·m, "
			  f"We={equTemTrack[i,3]:6.1f} rad/s")
	
	# 4. equWeTrack - 恒转速轨迹
	equWeTrack = result['equWeTrack']
	print(f"\n【4. 恒转速轨迹 (equWeTrack)】")
	print(f"  形状: {equWeTrack.shape}")
	print(f"  前3个点 [iq, id, Tem]:")
	for i in range(min(3, len(equWeTrack))):
		print(f"    [{i}] iq={equWeTrack[i,0]:7.3f} A, "
			  f"id={equWeTrack[i,1]:7.3f} A, "
			  f"Tem={equWeTrack[i,2]:6.1f} N·m, "
			  f"We={equWeTrack[i,3]:6.1f} rad/s")
	
	# 5. givenPoint - 给定工况点
	givenPoint = result['givenPoint']
	print(f"\n【5. 给定工况点 (givenPoint)】")
	print(f"  形状: {givenPoint.shape}")
	print(f"  值 [iq, id, Tem, We]:")
	print(f"    iq  = {givenPoint[0]:7.3f} A")
	print(f"    id  = {givenPoint[1]:7.3f} A")
	print(f"    Tem = {givenPoint[2]:6.1f} N·m")
	print(f"    We  = {givenPoint[3]:6.1f} rad/s")
	print(f"    calcTem = {calcTem(givenPoint[1],givenPoint[0])}")
	print(f"    calcUp = {calcUp(givenPoint[1],givenPoint[0],givenPoint[3])}")
	
	# ==================== 绘制We-Tem平面轨迹 ====================
	print("\n" + "="*80)
	print("绘制 We-Tem 平面轨迹图")
	print("="*80)
	
	fig, ax = plt.subplots(figsize=(12, 8))
	
	# 1. 绘制最大容量轨迹 (IsTrackAtMaxCapacity)
	if len(IsTrackAtMaxCapacity) > 0:
		We_maxCap = IsTrackAtMaxCapacity[:, 4]   # 第5列：We
		Tem_maxCap = IsTrackAtMaxCapacity[:, 3]  # 第4列：Tem
		ax.plot(We_maxCap, Tem_maxCap, 'b-', linewidth=2.5, 
				label='最大容量轨迹', 
				marker='o', markersize=4, 
				markevery=max(1, len(We_maxCap)//10))
	
	# 2. 绘制恒转速轨迹 (equWeTrack)
	if len(equWeTrack) > 0:
		ax.scatter(equWeTrack[:, 3], equWeTrack[:, 2], s=50, c='green', 
				  marker='s', label='恒转速轨迹', zorder=5)
	
	# 3. 绘制恒转矩轨迹 (equTemTrack)
	if len(equTemTrack) > 0:
		ax.scatter(equTemTrack[:, 3], equTemTrack[:, 2], s=50, c='red', 
				  marker='^', label='恒转矩轨迹', zorder=5)
	
	# 4. 绘制工况点 (givenPoint)
	We_point = givenPoint[3]
	Tem_point = givenPoint[2]
	ax.scatter([We_point], [Tem_point], s=200, c='black', 
			  marker='*', 
			  label=f'工况点 (We={We_point:.1f}, Tem={Tem_point:.1f})', 
			  zorder=6, edgecolors='yellow', linewidths=2)
	
	# ==================== 设置图表 ====================
	ax.set_xlabel('电角速度 We (rad/s)', fontsize=12, fontweight='bold')
	ax.set_ylabel('电磁转矩 Tem (N·m)', fontsize=12, fontweight='bold')
	ax.set_title('id=0/MTPA + FW 控制策略轨迹分析 - We-Tem 平面', 
				fontsize=14, fontweight='bold')
	ax.grid(True, alpha=0.3, linestyle='--')
	ax.legend(loc='best', fontsize=10, framealpha=0.9)
	
	plt.tight_layout()
	plt.show()
	
	print("\n" + "="*80)
	print("测试完成！")
	print("="*80)

print("✓ 测试函数 test_id0_mtpa_result_visualization 已定义")

In [ ]:
# ==================== 测试 calcId0MtpaTracks 函数 ====================
print("="*80)
print("测试 calcId0MtpaTracks 函数")
print("="*80)

# ==================== 设置测试参数 ====================
print("\n[步骤1] 设置测试参数...")
Tem_target = 1500    # 目标转矩 (N·m)
We_target = 500     # 目标电角速度 (rad/s)
loadType = 'constWe'  # 工况类型：恒转速负载

print(f"  • Tem_target = {Tem_target:.1f} N·m")
print(f"  • We_target  = {We_target:.1f} rad/s")
print(f"  • loadType   = '{loadType}'")

# ==================== 调用函数 ====================
print("\n[步骤2] 调用 generateNonFwResult 函数...")

# result = generateNonFwResult(controlType='mtpa',temTarget=Tem_target,weTarget=We_target,loadType=loadType)
result = generateFwResult(controlType='mtpa',temTarget=Tem_target,weTarget=We_target,loadType=loadType)
print("  ✓ 函数调用成功")

# ==================== 使用测试函数进行验证和可视化 ====================
print("\n[步骤3] 使用测试函数进行结果验证和可视化...")
test_id0_mtpa_result_visualization(result)

### 3.4 弱磁工况点的直观绘图展示

In [ ]:
# 定义函数：绘制给定工况点下的三条轨迹（IsTrack、equTemTrack、IsTrackInEquU）
def plotTracksForOperatingPoint(Tem_target: float, We_target: float,
								 mtpaTrack: np.ndarray, fw1Track: np.ndarray, mtpvTrack: np.ndarray,
								 UPmax: float, IPmax: float, id_demag: float,
								 equUPoints: int = 100, equTemPoints: int = 50,
								 figsize: tuple = (12, 10)):
	"""
	绘制给定工况点(Tem_target, We_target)下的三条轨迹
	
	参数:
		Tem_target: 目标转矩 (N·m)
		We_target: 目标电角速度 (rad/s)
		mtpaTrack: MTPA轨迹 (N×4)，格式：[[iq0, id0, Is0, Tem0], ...]
		fw1Track: 弱磁I区轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, We0], ...]
		mtpvTrack: MTPV轨迹 (N×5)，格式：[[iq0, id0, Is0, Tem0, We0], ...]
		UPmax: 最大相电压幅值 (V)
		IPmax: 最大相电流幅值 (A)
		id_demag: 去磁电流 (A)
		equUPoints: 等电压椭圆扫描点数
		equTemPoints: 等转矩线扫描点数
		figsize: 图形大小
	
	返回:
		tuple: (IsTrack, equTemTrack, IsTrackInEquU)
	"""
	
	print(f"\n{'='*70}")
	print(f"绘制给定工况点的三条轨迹")
	print(f"目标工况点: Tem_target = {Tem_target:.2f} N·m, We_target = {We_target:.2f} rad/s")
	print(f"{'='*70}")
	
	# 步骤1：获取完整的电流容量边界 IsTrack
	print(f"\n[步骤1] 获取完整电流容量边界 IsTrack...")
	mtpa_part = mtpaTrack[:, :2]  # 取MTPA的前两列 [iq, id]
	fw1_part = fw1Track[:, :2]    # 取弱磁I的前两列 [iq, id]
	mtpv_part = mtpvTrack[:, :2]  # 取MTPV的前两列 [iq, id]
	IsTrack = np.vstack([mtpa_part, fw1_part, mtpv_part])
	print(f"  ✓ IsTrack 计算完成，点数: {len(IsTrack)}")
	
	# 步骤2：计算等转矩线 equTemTrack
	print(f"\n[步骤2] 计算等转矩线 equTemTrack (Tem = {Tem_target:.2f} N·m)...")
	equTemTrack = calcIsTrackInEquTem(
		Tem_target=Tem_target,
		UPmax=UPmax,
		IPmax=IPmax,
		mtpaTrack=mtpaTrack,
		fw1Track=fw1Track,
		mtpvTrack=mtpvTrack,
		equTemPoints=equTemPoints
	)
	print(f"  ✓ equTemTrack 计算完成，点数: {len(equTemTrack)}")
	
	# 步骤3：计算等电压椭圆 IsTrackInEquU
	print(f"\n[步骤3] 计算等电压椭圆 IsTrackInEquU (We = {We_target:.2f} rad/s)...")
	IsTrackInEquU, Tem_max_at_We = calcIsTrackInEquU(
		We=We_target,
		mtpaTrack=mtpaTrack,
		fw1Track=fw1Track,
		mtpvTrack=mtpvTrack,
		UPmax=UPmax,
		IPmax=IPmax,
		id_demag=id_demag,
		equUPoints=equUPoints
	)
	print(f"  ✓ IsTrackInEquU 计算完成，点数: {len(IsTrackInEquU)}, 最大转矩: {Tem_max_at_We:.2f} N·m")
	
	# 步骤4：绘制三条轨迹
	print(f"\n[步骤4] 绘制轨迹图形...")
	fig = plt.figure(figsize=figsize)
	
	# 绘制电流圆限制（Is = IPmax）
	theta = np.linspace(0, np.pi/2, 100)
	id_circle = -IPmax * np.cos(theta)
	iq_circle = IPmax * np.sin(theta)
	plt.plot(id_circle, iq_circle, 'k--', linewidth=1.5, alpha=0.6, label=f'电流限制圈 (Is={IPmax:.0f}A)')
	
	# 绘制IsTrack（完整电流容量边界）
	plt.plot(IsTrack[:, 1], IsTrack[:, 0], 'b-', linewidth=2.5, label='最大容量输出电流轨迹 (IsTrack)', zorder=10)
	
	# 绘制等转矩线
	if len(equTemTrack) > 0:
		plt.plot(equTemTrack[:, 1], equTemTrack[:, 0], 'r-', linewidth=2.0, 
				label=f'等转矩线 (Tem={Tem_target:.2f} N·m)', zorder=9)
	
	# 绘制等电压椭圆
	if len(IsTrackInEquU) > 0:
		plt.plot(IsTrackInEquU[:, 1], IsTrackInEquU[:, 0], 'g-', linewidth=2.0, 
				label=f'等电压椭圆 (We={We_target:.2f} rad/s)', zorder=8)
	
	# 设置图形属性
	plt.xlabel('id (A)', fontsize=12)
	plt.ylabel('iq (A)', fontsize=12)
	plt.title(f'给定工况点的最大容量/恒转矩输出/恒转速输出电流轨迹\nTem={Tem_target:.2f} N·m, We={We_target:.2f} rad/s', 
			 fontsize=14)
	plt.grid(True, alpha=0.3)
	plt.axhline(y=0, color='k', linestyle='--', alpha=0.3, linewidth=0.5)
	plt.axvline(x=0, color='k', linestyle='--', alpha=0.3, linewidth=0.5)
	plt.legend(loc='upper right', fontsize=10)
	plt.axis('equal')
	plt.tight_layout()
	
	print(f"  ✓ 图形绘制完成！")
	
	return IsTrack, equTemTrack, IsTrackInEquU


# 测试函数
print("\n" + "="*70)
print("测试 plotTracksForOperatingPoint 函数")
print("="*70)

# 测试工况：转矩50 N·m，转速150 rad/s
Tem_test = 500.0   # N·m
We_test = 300.0   # rad/s

IsTrack_test, equTemTrack_test, IsTrackInEquU_test = plotTracksForOperatingPoint(
	Tem_target=Tem_test,
	We_target=We_test,
	mtpaTrack=mtpaTrack,
	fw1Track=fw1Track,
	mtpvTrack=mtpvTrack,
	UPmax=UPmax,
	IPmax=IPmax,
	id_demag=id_demag,
	equUPoints=100,
	equTemPoints=50,
	figsize=(12, 10)
)